# Anne Frank Diary Analysis
## By Mark McGown and Moses Boudourides

In [1]:
#Source https://archive.org/stream/AnneFrankTheDiaryOfAYoungGirl_201606/Anne-Frank-The-Diary-Of-A-Young-Girl_djvu.txt
import re
import nltk
import math
nltk.download('vader_lexicon')
import spacy
import random
import inflect
import datetime
import community
!pip install leidenalg
import leidenalg
import numpy as np
import igraph as ig
import pandas as pd
import networkx as nx
from pprint import pprint
from spacy import displacy
from matplotlib import colors
from operator import itemgetter
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib.pyplot import cm
from itertools import permutations
from scipy.ndimage.filters import gaussian_filter1d
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#How to get pygraphviz
#!wget https://anaconda.org/anaconda/pygraphviz/1.3/download/linux-64/pygraphviz-1.3-py36h14c3975_1.tar.bz2
#!tar xvjf pygraphviz-1.3-py36h14c3975_1.tar.bz2
#!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/

#!sudo pip install python-igraph
!sudo apt-get install graphviz libgraphviz-dev pkg-config
!sudo apt-get install python-pip python-virtualenv
!pip install pygraphviz

import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout

!python -m spacy download en_core_web_lg
import en_core_web_lg
nlp = en_core_web_lg.load()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 3.2MB 8.7MB/s 
  Created wheel for leidenalg: filename=leidenalg-0.7.0-cp36-cp36m-linux_x86_64.whl size=1107506 sha256=47ecfc5e3c941152bdb3aacde7179c2db386c0ee06ca112913c51858f7393550
  Stored in directory: /root/.cache/pip/wheels/29/55/48/5a04693a10f50297bcda23819ca23ab3470a61dd911851c8bd
Successfully built leidenalg


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
The following NEW packages will be installed:
  libgraphviz-dev libgvc6-plugins-gtk libxdot4
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 91.3 kB of archives.
After this operation, 425 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libxdot4 amd64 2.40.1-2 [15.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgvc6-plugins-gtk amd64 2.40.1-2 [18.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgraphviz-dev amd64 2.40.1-2 [57.3 kB]
Fetched 91.3 kB in 1s (149 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 7

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1bKPRap5G-ZO2FLrS8WjdpScA6O4tgKAL"}) 
downloaded.GetContentFile('TheDiaryOfAYoungGirl.txt')

## 1. Detecting Terms in Anne Frank's Diary

In [0]:
#Regular expressions to look for when a new diary entry is beginning
random.seed(1000)
r = '(?:SUNDAY|MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY), (?:JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER) \d{1,2}, 19\d{2}\s*\n'
date_indexes = []
#Open the file
with open("TheDiaryOfAYoungGirl.txt", encoding="utf8", errors='ignore') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
      # if '1943' in line:
      #   #Make a new diary entry for date
      if re.match(r, line):
            date_indexes.append((line.strip(), i))
    dia = {}
    #Build dictionary of the date indices and their entries
    for i in range(0, len(date_indexes) - 1):
        cur_idx = date_indexes[i][1] + 1
        next_idx = date_indexes[i + 1][1] - 1
        dia.update({date_indexes[i][0]: ''.join(lines[cur_idx:next_idx])})
#pprint(dia['FRIDAY, MARCH 17, 1944'])

In [0]:
dia

In [0]:
i = 0
dd = []
dto_list = []
for key in dia.keys():
    td = key.split(',')
    td = td[1]+td[2]
    td = td.lstrip()
    dto = datetime.datetime.strptime(td, '%B %d %Y')
    dto_list.append(dto)
    if i == 0:
        firstd = dto
        dd.append(0)
    else:
        dd.append((dto-firstd).days)
    i = i + 1
#Consecutive day count from the first entry
print(dd)

In [0]:
dic = {}
i = 0
#Build a dictionary of the days with their respective text
for key, value in dia.items():
    dic.update({dd[i] : value.replace('\n','')})
    i = i +1
pprint(dic[0])

In [0]:
p = inflect.engine()
d_tags = {}

#Remove these parts of existing entities, e.g. Dearest Kitty vs. Kitty
def clean_up(X,ex_ls):
    clean_l = ['Dearest','Dear','kittle','\'s','\\','The','the','didn\'t','hasn\'t','doesn\'t','every','8,1943','gize,'
               ,'MAY 7,1944','wheeled','My','wasn\'t','COMMENT','14g','I\'m','couldn\'t','Great','Old','future','looked'
               ,'willi','de\'','d\'']
    for clean in clean_l:
        if (clean in X) and (X not in ex_ls):
            X = X.replace(clean,'')
            X = X.lstrip()
            X = X.rstrip()
    if p.singular_noun(X) and (X not in ex_ls):
        X = p.singular_noun(X)
    return X

#Reg expressions to detect nlp entities and reclassify them to standardize
def standardize(X,ex_ls):
    #Things I'd like to standardize to aggregate better
    rep = {'^anne':'Anne','^van d':'van D.','home guard':'Home Guard','^gies &':'Gies & Co.'
           ,'secret annex':'Annex','^bep':'Bep','^miep':'Miep','^margot':'Margot'
          ,'^peter':'Peter','hanneli':'Hanneli','hiller':'Hitler','^g.z. e. s.':'G.Z.',
          '^olga meyen- dorff' : 'Olga Meyen-dorff','^ursul':'Ursula','kitty':'Kitty','das schon':'das schon'}
    for key,value in rep.items():
        if (re.match(key,X.replace('"','').lower())) and (X not in ex_ls):
            X = rep[key]
    return X

#Make entity tags of nlp for each diary or, later, sentence entry
def make_entities(dic,keep_l):
    for key, value in dic.items():
        arr = []
        doc = nlp(value.replace('\n',''))
        #Keep these types of nlp entities
        #Typo/model error + german corrections
        drop_t = ['it','you\'re','he','his','her','ei ght','bd\'"dl','she','i\'m','there','who','wasn\'t','didn\'t','that','here','we\'ll'
              ,'let','what','i','son','he\'ll','re','wouldn\'t','y\'d','ridicu lou','lady','one','fed','i\'d','\'oh','you\'d'
              ,'s','shh','tranquthty','civthzed','now fifteen','hadn\'t','sun','\'i','she\'ll','man','my hand','saw'
              ,'\'remem ber','re\'re','you\'ve','you\'ll','shuffiing','he\'d','bye','nitely','much tidier','semidarknes'
              ,'you','te','mean-','conceited','my head','we\'d','i\'d lain','you\'ve','mothproofed'
              ,'\'that','famthar','noth','mothjr','m r','clodiesline','diat','togedier','algebra','fadier','entsetzlich'
              ,'nie zu ersetzen','nothina','\'that','beina','future','secredy','hand','writina','dealina','enouah','runnina'
              ,'unfortu','house','jun','\'he','brrr','i. boy','mally','dread','pst','chin','phy','iii','head'
              ,'cinema ater','sore throat','ketde','baker','seething','silent','tired','sister','lady','balli'
              ,'ding-dong','sohn','ron','mor','himmelhoch jauchzend','zu tode','der mann','jan gy','das liebe','das schone'
              ,'urn gotte','keg','eastern','du spritzt schon','quicksilver','espe','brea','miep, bep','gy','du bist doch eine'
              ,'ich mach\'','ruddy','kitty, bep','ifhe','pedes apostolorum','das schon','1','everYthing']

        #Things inflect library handles poorly or to exclude from touching
        ex_ls = ['Swiss','William Louis','Theseus','Zeus','Oedipus','Peleus','Orpheus','Hercules','Oasis','Maria Theresa','Myron'
                ,'Phidias','Annex','Trees','Broks','Dutch Sasas','Kuperus']

        for X in doc.ents:
            s1 = X.text
            if (X.label_ in keep_l):
                s1 = clean_up(s1,ex_ls)
                s1 = standardize(s1,ex_ls)
                if (s1.lower() not in drop_t) and (s1):
                    arr.append((s1, X.label_))
        d_tags[key] = arr
    return d_tags

keep_l = ['PERSON','NORP','PRODUCT','ORG']
d_tags = make_entities(dic,keep_l)
pprint(d_tags[0])

### 2. Dictionary of Attributes of Terms (NLP Entities)

In [0]:
chars = []
full_tags = []
tag_count_dic = {}
j = 0
for key, value in d_tags.items():
    i = 0
    tags = []
    tag_count_dic[dto_list[j]] = {}
    for elem in value:
        full_tags.append(value[i])
        chars.append(value[i][0])
        tags.append(value[i][1])
        i += 1
    for tag in set(tags):
        tag_count_dic[dto_list[j]][tag] = tags.count(tag)
    j += 1

In [0]:
tag_dic = {}
node_dic = {}
drop = ''
for tag in full_tags:
    if tag[0] not in tag_dic.keys():
        tag_dic[tag[0]] = [tag[1]]
    else:
        tag_dic[tag[0]].append(tag[1])

for key in tag_dic.keys():
  node_dic[key] = {}
  if key not in drop:
    node_dic[key]['tag'] =  max(set(tag_dic[key]), key = tag_dic[key].count) 

node_dic

## 3. Time Series of Attributes

In [0]:
dft = pd.DataFrame(tag_count_dic)
dft = dft.fillna(0).transpose()
plt.subplots(figsize=(18,12))
columns = list(dft)
i = 0
cmap = cm.get_cmap('tab10')
for column in columns:
    y = gaussian_filter1d(dft[column],sigma=2.5)
    x = list(tag_count_dic.keys())
    plt.fill_between(x,y, color=cmap.colors[i], alpha=0.4)
    plt.plot(x,y,color=cmap.colors[i],label=column)
    i += 1

plt.xlabel('Date',size=40)
plt.ylabel('NLP Tag Count',size=40)
plt.title('Tag Count Over Time',size=60)
plt.ylim(ymin=0)
plt.xlim(xmax=max(x),xmin=min(x))
plt.legend(loc='upper right',prop={'size':14})
plt.tick_params(axis='both', which='major', labelsize=20)
plt.tick_params(axis='x', which='major',rotation=-45)
#plt.setp(plt.xaxis.get_majorticklabels(), rotation=-45 ) 
#lines = dft.plot.line(figsize=(18,12))

### 4. Diary Co-Occurrences

In [0]:
unique_chars = list(set(chars))
unique_full_tags = list(set(full_tags))
print(sorted(unique_chars))

In [0]:
df = pd.DataFrame(columns = unique_chars, index = unique_chars)
df[:] = int(0)

for key,value in d_tags.items():
    arr = []
    for elem in d_tags[key]:
        arr.append(elem[0])
    arr.append(elem[0])
    for char1 in unique_chars:
        for char2 in unique_chars:
            if char1 in arr and char2 in arr:
                df[char1][char2] += 1
                df[char2][char1] += 1
df.head()

### 5. Sentential Co-Occurrences

In [0]:
full_txt = ''
sent_dic = {}
sentences = []
i = 0
for z,v in dia.items():
    v = v.replace('\n','')
    full_txt += v
    dia[z] = v

In [0]:
dia

In [0]:
def make_sentences(full_txt):
    doc = nlp(full_txt)
    sent_list = []
    for i, token in enumerate(doc.sents):
        for entity in unique_chars:
            if entity in token.text:
                sent_list.append(token.text)
                break
    return sent_list

In [0]:
from wordcloud import WordCloud

plt.figure(figsize=(12,6))

filtered_txt = ''
last_word = ''
for word in full_txt.split():
  if (word in unique_chars) and (word != last_word):
    filtered_txt += " " + word
    last_word = word

# Generate a word cloud image
wordcloud = WordCloud(background_color="black",width=800,height=400).generate(filtered_txt)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Wordcloud of Anne Frank\'s Diary',size=40)
plt.show()

In [0]:
i = 0
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
overall_sent_dic = {}
for z,v in dia.items():
  l1 = make_sentences(v)
  for sentence in l1:
    overall_sent_dic[i] = {}
    overall_sent_dic[i]['sentence'] = sentence
    overall_sent_dic[i]['date'] = z
    i += 1
overall_sent_dic[0]

In [0]:
sentiments = []
sent_list = make_sentences(full_txt)

In [0]:
i = 0
sent_dic = {}
while i < len(sent_list):
    sent_dic[i] = sent_list[i]
    i += 1
sent_tags = make_entities(sent_dic,keep_l)

sent_tags[0]

### 6. Sentiment of Sentential Co-Occurrences

In [0]:
for key in sent_tags.keys():
    sentiment = sid.polarity_scores(sent_list[key])
    ent_list = []
    for tag in sent_tags[key]:
        ent_list.append(tag[0])
    sentiments.append((ent_list,sentiment))
sentiments[0:5]

In [0]:
unique_chars.append('Mother')
unique_chars.append('Father')
unique_chars.append('Grandma')
unique_chars.append('Mr. van Daan')
unique_chars.append('Mr. van D.')
unique_chars.append('Mrs. van Daan')
unique_chars.append('Mrs. van D.')

In [0]:
for z,v in overall_sent_dic.items():
  unique_list = []
  word_list = [i.lower().strip(',') for i in v['sentence'].split()]
  word_list = [i.split('\'')[0] for i in word_list]
  for char in unique_chars:
    for w in word_list:
      if (char.lower()==w):
        unique_list.append(char)
    if (' ' in char) and (char in v['sentence']):
      unique_list.append(char)
  for word in unique_list:
    for word2 in unique_list:
      if ' ' in word2:
        word3 = word2.split(' ')
        for word4 in word3:
          if (word == word4) and (word in unique_list):
            unique_list.remove(word)
  overall_sent_dic[z]['sentiment'] = sid.polarity_scores(v['sentence'])['compound']
  overall_sent_dic[z]['cooccurrences'] = list(set(unique_list))
overall_sent_dic[1]

In [0]:
df_sent = pd.DataFrame.from_dict(overall_sent_dic).transpose()
df_sent.index.name = 'SentID'
df_sent.head()

In [0]:
columns=['Date','Persons','Average Sentiment','Sentential Sentiment','Text']
df_sent2 = pd.DataFrame(columns=columns)

tup = []
text_list = []
dto,last_dto = 0,0
for idx, (_, value) in enumerate(df_sent.iterrows()):
    date = value.date
    td = date.split(',')
    td = td[1]+td[2]
    td = td.lstrip()
    dto = datetime.datetime.strptime(td, '%B %d %Y')
    if ((dto != last_dto) or (idx==len(df_sent)-1)) and len(tup):
      df_sent2.loc[last_idx] = pd.Series({'Date':last_dto, 'Persons':list(set(x for l in [i for (i,j) in tup] for x in l)), 'Average Sentiment':sum([i[1] for i in tup])/len([i[1] for i in tup]), 'Sentential Sentiment':tup,'Text':text_list})
      text_list = []
      tup = []
    last_dto = dto
    last_idx = idx
    if len(value.cooccurrences) > 1:
      tup.append((value.cooccurrences,value.sentiment))
      text_list.append(value.sentence)
df_sent2

In [0]:
sentiments2 = []
for element in sentiments:
    ent_comb = list(permutations(element[0],2))
    for ent_tup in ent_comb:
      sentiments2.append((ent_tup,element[1]['compound']))
sentiments2[0:25]

In [0]:
#Build dictionary of sentiments observed per edge
sent_dic = {}
for element in sentiments2:
    if element[0] not in sent_dic:
        sent_dic[element[0]] = [element[1]]
    else:
        sent_dic[element[0]].append(element[1])

In [0]:
duplicates = []
for z in sent_dic:
  if ((z[1],z[0]) in sent_dic) and ((z[1],z[0]) not in duplicates):
    duplicates.append(z)

for duplicate in duplicates:
  del sent_dic[duplicate]

len(sent_dic)

## 7. Graph Plots

In [0]:
def make_edges(df):
    #Find edge values to determine max
    edge_values = []
    edge_list = {}
    for index, row in df.iterrows():
        i = 0
        for col in row:
            if col > 0 and (index != row.index[i]):
                edge_values.append(col)
            i += 1
    
    #Get max edge weight as percentage of that max edge value
    for index, row in df.iterrows():
        i = 0
        for col in row:
            #Remove edges with no weight, self-referential edges, and edges already added with inverse keys
            if col > 0 and (index != row.index[i]) and ((df.columns[i], index) not in list(edge_list.keys())):
                weight = float(col)/max(edge_values)
                edge_list[(index, df.columns[i])] = weight
            i += 1
    return edge_list

In [0]:
def drop_nodes(df,drop_list):
  df = df.drop(columns=drop_list)
  df = df.drop(drop_list)
  return df

In [0]:
def make_G(df,cooccurrence,filter,node_dic,sent_dic,sentiments2,ego='',drop=''):

  if drop:
    df2 = drop_nodes(df,[drop])
  else:
    df2 = df.copy()

  df3 = df2.copy()
  if filter:
    df3[df3 < filter] = 0

  edge_list = make_edges(df3)

  if cooccurrence == 'sentences':
    remove_edges = []
    for edge in edge_list:
      if (edge not in sent_dic) and ((edge[1],edge[0]) not in sent_dic):
        remove_edges.append(edge)
    for edge in remove_edges:
      del edge_list[edge]

  node_list = set([i for t in edge_list for i in t])

  G = nx.Graph()
  G.add_nodes_from(node_list)
  G.add_edges_from(edge_list)

  if ego:
    G = nx.ego_graph(G, ego)

  n_dic = {}
  e_dic = {}
  if cooccurrence=='diary':
    for node in G.nodes():
      n_dic[node] = {}
      n_dic[node]['size'] = df3[node].sum()/2
      n_dic[node]['tag'] = node_dic[node]['tag']
    for edge in G.edges():
      if edge in edge_list:
        e_dic[edge] = {}
        e_dic[edge]['width'] = edge_list[edge]
      else:
        e_dic[edge] = {}
        e_dic[edge]['width'] = edge_list[(edge[1],edge[0])]

  else:
    for node in G.nodes():
      count = 0
      n_dic[node] = {}
      for sent in sent_dic:
        if (node == sent[0]) or (node == sent[1]):
          count += 1
      n_dic[node]['size'] = count
      n_dic[node]['tag'] = node_dic[node]['tag']
    for edge in G.edges():
      if edge in sent_dic:
        e_dic[edge] = {}
        e_dic[edge]['sentiment'] = round(np.mean(sent_dic[edge]),2)
        e_dic[edge]['width'] = [i[0] for i in sentiments2].count(edge)
      elif (edge[1],edge[0]) in sent_dic:
        e_dic[edge] = {}
        e_dic[edge]['sentiment'] = round(np.mean(sent_dic[(edge[1],edge[0])]),2)
        e_dic[edge]['width'] = [i[0] for i in sentiments2].count((edge[1],edge[0]))

  return G,n_dic,e_dic

### 7a. Full Graph, Diary Co-Occurrent

In [0]:
plt.subplots(figsize=(20,20))

node_scalar = 0.7
edge_scalar = 100

G_everything,n_dic,e_dic = make_G(df,'diary',0,node_dic,sent_dic,sentiments2)

pos = graphviz_layout(G_everything)

nx.draw_networkx_edges(G_everything, pos, width=[e_dic[i]['width']*edge_scalar for i in G_everything.edges], edge_color='b', style='solid', alpha=0.05)
nx.draw_networkx_nodes(G_everything, pos, with_labels=False,node_list=n_dic, font_size = 18, font_weight = 'bold',node_size=[n_dic[i]['size']*node_scalar for i in G_everything.nodes])
nx.draw_networkx_labels(G_everything, pos, with_labels=True,node_list=n_dic, font_size = 12, font_weight = 'bold')

plt.title('Diary Co-Occurrence of All Terms',size=40)

## 7b. Frequent Diary Co-Occurrent Terms (Anne Removed)

In [0]:
import math
plt.subplots(figsize=(10,10))

node_scalar = 3
edge_scalar = 10

G,n_dic,e_dic = make_G(df,'diary',14,node_dic,sent_dic,sentiments2,'','Anne')

pos = graphviz_layout(G)

nx.draw_networkx_edges(G, pos, width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges], edge_color='b', style='solid', alpha=0.8)
nx.draw_networkx_nodes(G, pos, node_list=n_dic, font_size = 12, font_weight = 'bold',node_size=[n_dic[i]['size']*node_scalar for i in G.nodes])
nx.draw_networkx_labels(G, pos, with_labels=True,node_list=n_dic, font_size = 12, font_weight = 'bold')

plt.title('Diary Co-Occurrence of Filtered Terms (Anne Removed)',size=20)

## 7c. Sentiment of Sentential Co-Occurrent Terms (Anne Removed)

In [0]:
import matplotlib

plt.subplots(figsize=(10,10))

node_scalar = 80
edge_scalar = 5

G,n_dic,e_dic = make_G(df,'sentences',8,node_dic,sent_dic,sentiments2,'','Anne')

pos = graphviz_layout(G,'neato')

sentiments = [e_dic[i]['sentiment'] for i in G.edges]

if min(sentiments) >= 0:
  min_sent = -1
else:
  min_sent = min(sentiments)

if max(sentiments) <= 0:
  max_sent = 1
else:
  max_sent = max(sentiments)
from matplotlib.pyplot import cm
cmap = cm.get_cmap('viridis_r')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent, vmax=max_sent)

sentiment_colors = [cmap(norm(i)) for i in sentiments]

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm) #, orientation='horizontal')
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

nx.draw(G, pos, with_labels=True,node_list=n_dic, font_size = 12, font_weight = 'bold',node_size=[n_dic[i]['size']*node_scalar for i in G.nodes],width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges],edge_color=sentiment_colors)
plt.title('Sentential Co-Occurrences of Filtered Terms\n(Anne Removed) w/ Sentiment (1944)',size=20)

In [0]:
def avg_sent(G,e_dic):
  avsen_dic = {}
  for node in G.nodes:
    avsen_dic[node] = {}
    sen_lis = []
    tot_width = []
    for edge in e_dic.keys():
      if (node in edge[0]) or (node in edge[1]):
        sen_lis.append(e_dic[edge]['sentiment']*e_dic[edge]['width'])
        tot_width.append(e_dic[edge]['width'])
    avsen_dic[node]['sentiment'] = sum(sen_lis)/len(sen_lis)
    avsen_dic[node]['weighted_sentiment'] = sum(sen_lis)/(len(sen_lis)*sum(tot_width))
  return avsen_dic

sen_dic1 = avg_sent(G,e_dic)

def bar_sent(sen_dic1):
  from matplotlib.pyplot import cm
  sen_dic2 = {}
  for node in sen_dic1.keys():
    sen_dic2[node] = sen_dic1[node]['weighted_sentiment']
  D = {k: v for k, v in sorted(sen_dic2.items(), key=lambda item: item[1])}

  sentiments = D.values()

  if min(sentiments) >= 0:
    min_sent = -1
  else:
    min_sent = min(sentiments)

  if max(sentiments) <= 0:
    max_sent = 1
  else:
    max_sent = max(sentiments)

  cmap = cm.get_cmap('viridis_r')

  norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent, vmax=max_sent)

  sentiment_colors = [cmap(norm(i)) for i in sentiments]

  fig, ax = plt.subplots()

  # Add some text for labels, title and custom x-axis tick labels, etc.
  ax.set_ylabel('Average Sentiment',size=18)
  ax.set_title('Weighted Sentiment by Node (1944)',size=24)
  ax.legend()

  fig.tight_layout()

  fig.set_figheight(10)
  fig.set_figwidth(8)

  #sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
  #sm.set_array([])
  #cbar = plt.colorbar(sm) #, orientation='horizontal')
  #cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

  plt.bar(range(len(D)), list(D.values()), align='center',color=sentiment_colors)
  plt.xticks(range(len(D)), list(D.keys()))

  plt.xticks(rotation=90)
  plt.axhline(y=0, color='k', linestyle='-')

  plt.show()
bar_sent(sen_dic1)

In [0]:
pd.DataFrame.from_dict(sen_dic1).T

In [0]:
test_b = pd.DataFrame.from_dict(overall_sent_dic).T
test_b.head(30)

# for d in set(test_b['date']):
#   print(d)

In [0]:
test_b = test_b[["cooccurrences","date","sentiment","sentence"]]
test_b

In [0]:
from scipy.stats import linregress

def sent_slope(e_dic,overall_sent_dic):
  slope_dic = {}
  for edge in e_dic.keys():
    #slope_dic[edge] = {}
    days = []
    sen_data = []
    for k in overall_sent_dic.keys():
      if (edge[0] in overall_sent_dic[k]['cooccurrences']) and (edge[1] in overall_sent_dic[k]['cooccurrences']):
        days.append(k)
        sen_data.append(overall_sent_dic[k]['sentiment'])
    if days and sen_data:
      slope = round(linregress(days,sen_data).slope,4)
    else:
      slope = 0
    
    if math.isnan(slope):
      slope = 0

    slope_dic[edge] = slope*1000

  return slope_dic

In [0]:
from matplotlib.pyplot import cm

plt.subplots(figsize=(20,10))

node_scalar = 90
edge_scalar = 3

pos = graphviz_layout(G,'dot')

sent_slope_dic = sent_slope(e_dic,overall_sent_dic)

sentiments = [sent_slope_dic[i] for i in G.edges]

if min(sentiments) >= 0:
  min_sent = -1
else:
  min_sent = min(sentiments)

if max(sentiments) <= 0:
  max_sent = 1
else:
  max_sent = max(sentiments)

cmap = cm.get_cmap('viridis_r')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=-1, vmax=1)

sentiment_colors = [cmap(norm(i)) for i in sentiments]

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
cbar = plt.colorbar(sm) #, orientation='horizontal')
cbar.ax.set_ylabel('Slope of Sentiment',size=30,rotation=270,labelpad=30)

nx.draw(G, pos, with_labels=True,node_list=n_dic, font_size = 12, font_weight = 'bold',node_size=[n_dic[i]['size']*node_scalar for i in G.nodes],width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges],edge_color=sentiment_colors)
plt.title('Sentential Co-Occurrences of Filtered Terms\n(Anne Removed) w/ Sentiment Slope',size=20)

In [0]:
def avg_sent_slope(G,sent_slope_dic,e_dic):
  avsen_dic = {}
  for node in G.nodes:
    avsen_dic[node] = {}
    sen_lis = []
    tot_width = []
    for edge in G.edges:
      if (node in edge[0]) or (node in edge[1]):
        sen_lis.append(sent_slope_dic[edge]*e_dic[edge]['width'])
        tot_width.append(e_dic[edge]['width'])
    print(node,sen_lis)
    avsen_dic[node]['sentiment'] = sum(sen_lis)/len(sen_lis)
    avsen_dic[node]['weighted_sentiment'] = sum(sen_lis)/(len(sen_lis)*sum(tot_width))
  return avsen_dic

sen_dic1 = avg_sent_slope(G,sent_slope_dic,e_dic)

def bar_sent(sen_dic1):
  from matplotlib.pyplot import cm
  sen_dic2 = {}
  for node in sen_dic1.keys():
    sen_dic2[node] = sen_dic1[node]['weighted_sentiment']
  D = {k: v for k, v in sorted(sen_dic2.items(), key=lambda item: item[1])}

  sentiments = D.values()

  if min(sentiments) >= 0:
    min_sent = -1
  else:
    min_sent = min(sentiments)

  if max(sentiments) <= 0:
    max_sent = 1
  else:
    max_sent = max(sentiments)

  cmap = cm.get_cmap('viridis_r')

  norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent, vmax=max_sent)

  sentiment_colors = [cmap(norm(i)) for i in sentiments]

  fig, ax = plt.subplots()

  # Add some text for labels, title and custom x-axis tick labels, etc.
  ax.set_ylabel('Average Sentiment',size=18)
  ax.set_title('Weighted Sentiment by Node (1944)',size=24)
  ax.legend()

  fig.tight_layout()

  fig.set_figheight(10)
  fig.set_figwidth(8)

  #sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
  #sm.set_array([])
  #cbar = plt.colorbar(sm) #, orientation='horizontal')
  #cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

  plt.bar(range(len(D)), list(D.values()), align='center',color=sentiment_colors)
  plt.xticks(range(len(D)), list(D.keys()))

  plt.xticks(rotation=90)
  plt.axhline(y=0, color='k', linestyle='-')

  plt.show()
bar_sent(sen_dic1)

### 7d. Clustering Functions for Graph Plots

In [0]:
nlp_misclassifieds = {'Miep':'PERSON','Moortje':'PERSON'}
for node in nlp_misclassifieds:
  if node in node_dic:
    node_dic[node]['tag'] = nlp_misclassifieds[node]

In [0]:
def make_clusters_async(G,clusters=5):
    from networkx.algorithms import community
    from matplotlib import colors
    import itertools
    communities = list(nx.community.asyn_fluidc(G, clusters))
    i = 0
    node = {}
    cmap = cm.get_cmap('Set3')
    node_colors = []
    for entity in sorted(G.nodes):
        for n, com in enumerate(list(communities)):
            if entity in com:
                color = colors.to_hex(cmap.colors[n])
        node_colors.append(color)
    return node_colors

In [0]:
test_nodes = []
tag_to_train = 'NORP'
for node in G.nodes:
  if node in node_dic.keys():
    if node_dic[node]['tag'] == tag_to_train:
      test_nodes.append(node)
test_nodes

In [0]:
tags_that_exist = []
for node in G.nodes:
  tags_that_exist.append(n_dic[node]['tag'])
tags_that_exist = set(tags_that_exist)
tags_that_exist

In [0]:
i = 0
z = 0
h = 0
clusters = len(tags_that_exist)
while (i != (len(test_nodes)-h)) or (min(color_l) < (4-h)):
  node_colors = make_clusters_async(G,clusters)
  j = 0
  color_dic = {}
  while j < len(n_dic):
      color_dic[list(n_dic)[j]] = node_colors[j]
      j += 1
  max_color = {}
  for tnode in test_nodes:
      count = 0
      for tnode2 in test_nodes:
          if tnode != tnode2:
              if color_dic[tnode] == color_dic[tnode2]:
                  count += 1
          max_color[tnode] = count
  i = max(max_color.values())
  color_l = []
  for color in set(node_colors):
    color_l.append(node_colors.count(color))
  if z == 1000:
    h += 1
    z = 0
  z += 1

  if h > 4:
    break

x = 0
color_comm_membership = {}
for value in set(color_dic.values()):
  cluster = []
  for node in n_dic:
    if color_dic[node] == value:
      cluster.append(node)
  color_comm_membership[x] = cluster
  x += 1
i

In [0]:
cmap = cm.get_cmap('Set3')
for i in color_comm_membership:
  for node in color_comm_membership[i]:
    n_dic[node]['fluid_color'] = cmap.colors[i]
n_dic

In [0]:
import math
def make_coordinates(num,coord_scale=9):
  angle = 2*math.pi/num#+random.uniform(0, 1)
  angle_list = []
  i = 0
  while i < num:
    angle_list.append(i*angle)
    i += 1
  coordinates = []
  coordinates.append((0,-1*coord_scale))
  j = 1
  while j < len(angle_list):
    coordinates.append((round(math.cos(angle_list[j]-math.pi/2)*coord_scale,2),round(math.sin(angle_list[j]-math.pi/2)*coord_scale,2)))
    j += 1
  return coordinates
make_coordinates(len(tags_that_exist))

In [0]:
def sort_position_list(pos,x_dic,x_list,nodes=n_dic):
  pos_list = []
  num_nodes = []
  i = 0
  for l in x_list:
    position_dic = {}
    for node in nodes:
      if x_dic[node]['tag'] == l:
        position_dic[node] = pos[node]
    if position_dic:
      pos_list.append(position_dic)

  def getNumKeys(dict1):
    return len(dict1.keys())
  pos_list=sorted(pos_list, key=getNumKeys)[::-1]
  return pos_list

In [0]:
pos_list = sort_position_list(pos,n_dic,tags_that_exist)
coordinates = make_coordinates(len(tags_that_exist))
def sort_coord(coordinates,pos_list):
  i=0
  y_list = []
  miny,maxy = (), ()
  coord_list = []
  tag_list = []
  tag_type = ''
  for coord in coordinates:
    tag_type = n_dic[list(pos_list[i].keys())[0]]['tag']
    y_list.append(coord[1])
    tag_list.append((tag_type,len(pos_list[i].keys())))
    i += 1

  def getKey(item):
    return item[1]
  tag_list=sorted(tag_list, key=getKey)[::-1]

  for coord in coordinates:
    if coord[1] == max(y_list):
      maxy = coord
    if coord[1] == min(y_list):
      miny = coord
  coord_list.append(miny)
  coord_list.append(maxy)
  for coord in coordinates:
    if (coord != miny) and (coord != maxy):
      coord_list.append(coord)
  return coord_list,tag_list

coord_list,tag_list = sort_coord(coordinates,pos_list)
coord_list

In [0]:
interim_tag = ['']*len(tags_that_exist)
def getIndexTuple(tup_list,tag):
  i = 0
  for tup in tup_list:
    index = i
    if tup[0] == tag:
      break
    i += 1
  return index
    
for tag in tags_that_exist:
  ind = getIndexTuple(tag_list,tag)
  interim_tag[ind]=tag

tags_that_exist = interim_tag
tags_that_exist

In [0]:
def circular_position(x_list,coord_list,pos,pos_list,x_dic):
  cir_pos_list = []
  #k = 0.8
  i = 0
  radius_scalar = 0.25 #2.5/coord_scale
  for l in x_list:
    G1 = nx.Graph()
    node_list = []
    for node in G.nodes:
      if x_dic[node]['tag'] == l:
        node_list.append(node)
    G1.add_nodes_from(node_list)
    #pos = nx.spring_layout(G1, k, iterations=17)
    pos = nx.circular_layout(G1,scale=len(G1.nodes)*radius_scalar)
    #print(pos)
    for key in pos.keys():
      pos[key] = [pos[key][0]+coord_list[i][0],pos[key][1]+coord_list[i][1]]
    cir_pos_list.append(pos)
    i += 1
  return cir_pos_list

In [0]:
pos2 = {}
cir_pos_list = circular_position(tags_that_exist,coord_list,pos,pos_list,n_dic)
i=0
while i < len(cir_pos_list):
  for key in cir_pos_list[i].keys():
    pos2[key] = cir_pos_list[i][key]
  i += 1
pos2

In [0]:
!pip install circle-fit
import circle_fit as cf

def show_circle_layout(cir_pos_list,min_radius=1):
  cmap = cm.get_cmap('Set3')
  circle_list = []
  xrange = []
  yrange = []
  rmax = 0
  fig, ax = plt.subplots(figsize=(10,10))
  ax.set_aspect("equal")
  i = 0
  for element in cir_pos_list:
    points = list(element.values())
    if len(points) > 1:
      xc,yc,r,_ = cf.least_squares_circle(points)
    else:
      xc,yc,r = pos2[list(element.keys())[0]][0],pos2[list(element.keys())[0]][1],min_radius
    #circle = plt.Circle((xc, yc), r, color='r')
    #fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplt
    # (or if you have an existing figure)
    #fig = plt.gcf()
    #ax = fig.gca()
    if r > rmax:
      rmax = r
    circle_list.append((xc,yc,r))
    xrange.append(xc-r)
    xrange.append(xc+r)
    yrange.append(yc-r)
    yrange.append(yc+r)
    c=plt.Circle((xc,yc),r,color=cmap.colors[i])
    ax.add_artist(c)
      # Show/save figure as desired.
  plt.xlim(min(xrange),max(xrange))
  plt.ylim(min(yrange),max(yrange))
  plt.show()
  return circle_list,rmax

  #fig.savefig('plotcircles.png')
circle_list,rmax = show_circle_layout(cir_pos_list)

In [0]:
tag_cir_dic = {}
i = 0
for tag in tags_that_exist:
  tag_cir_dic[tag] = circle_list[i]
  i += 1
tag_cir_dic

In [0]:
circle_list

In [0]:
from sympy.solvers import solve
from sympy import Symbol
def find_xydelta(circle_list,rmax,offset=2):
  i = 0
  ref_circle = ()
  xychange = []
  for circle in circle_list:
    r = circle[2]
    if (r == rmax) and (i == 0):
      ref_circle = circle
      i = 1
  i = 0
  for circle in circle_list:
    x_cir,y_cir,r = circle[0],circle[1],circle[2]
    x_ref,y_ref,r_ref = ref_circle[0],ref_circle[1],ref_circle[2]
    x_move = 0
    y_move = 0
    if (r == rmax) and (i == 0):
      i = 1
    #elif abs(round(y_cir/y_ref,3)) == 1:
    #  y_move = y_cir-y_ref + offset + r_ref
    elif round(x_cir/x_ref,3) == 1:
      if y_cir > y_ref:
        y_move = y_cir-y_ref-r_ref-offset
      elif y_ref > y_cir:
        y_move = -(y_ref-y_cir-r_ref-offset)
    elif round(y_cir/y_ref,3) == 1:
      if x_cir > x_ref:
        x_move = x_cir-x_ref-r_ref-offset
      elif x_ref > x_cir:
        x_move = -(x_ref-x_cir-r_ref-offset)
    else:
      m = (y_cir-y_ref)/(x_cir-x_ref)
      b = y_cir-m*x_cir
      x = Symbol('x')
      ref_answer = solve((x - x_ref)**2+(m*x+b-y_ref)**2-(r_ref+offset)**2, x)
      x_ref_ans = 0
      for a in ref_answer:
        if (x_ref < a < x_cir) or (x_ref > a > x_cir):
          x_ref_ans = a
        y_ref_ans = x_ref_ans*m+b
      answer = solve((x - x_cir)**2+(m*x+b-y_cir)**2-(r)**2, x)
      x_ans = 0
      for a in answer:
        if (x_ref < a < x_cir) or (x_ref > a > x_cir):
          x_ans = a
        y_ans = x_ans*m+b
      x_move = round(x_ans - x_ref_ans,2)
      y_move = round(y_ans - y_ref_ans,2)
    xychange.append((-x_move,-y_move))
  return xychange

xychange = find_xydelta(circle_list,rmax)
xychange

In [0]:
def centralize_circles(cir_pos_list,xychange):
  i = 0
  pos_list2 = []
  while i < len(cir_pos_list):
    for key in cir_pos_list[i].keys():
      new_node_list2 = {}
      new_node_list2[key] = [cir_pos_list[i][key][0]+xychange[i][0],cir_pos_list[i][key][1]+xychange[i][1]]
      pos_list2.append(new_node_list2)
    i += 1
  pos_list2

  final_pos_dic = {}
  i=0
  while i < len(pos_list2):
    for key in pos_list2[i].keys():
      final_pos_dic[key] = pos_list2[i][key]
    i += 1
  return final_pos_dic

final_pos_dic = centralize_circles(cir_pos_list,xychange)
final_pos_dic

In [0]:
def add_center_circles(xychange,circle_list,names):
  i = 0
  G2 = nx.Graph()
  pos_dic = {}
  center_dic = {}
  sizes = []
  r_dic = {}
  node_colors = []
  cmap = cm.get_cmap('Set3')

  while i < len(circle_list):
    name = names[i]
    x = circle_list[i][0]+xychange[i][0]
    y = circle_list[i][1]+xychange[i][1]
    r_dic[name] = circle_list[i][2]*100000
    pos_dic[name] = [x,y]
    center_dic[name] = circle_list[i][2]
    node_colors.append(cmap.colors[i])
    i += 1
  
  G2.add_nodes_from(center_dic)
  for node in G2.nodes:
    sizes.append(r_dic[node])

  return G2,pos_dic,sizes,node_colors

In [0]:
def overall_graph(node_dic,edge_dic,tag_cir_dic,node_scalar=10000,edge_scalar=20):
  cmap = cm.get_cmap('Set3')
  tag_node_dic = {}
  tag_edge_dic = {}
  tag_pos_dic = {}
  i = 0
  for tag in set(tag_cir_dic):
    node_num = 0
    tag_node_dic[tag] = {}
    size_l = []
    for node in node_dic:
      if node_dic[node]['tag'] == tag:
        node_num += 1
        size_l.append(node_dic[node]['size'])
    tag_node_dic[tag]['node_num'] = node_num
    tag_node_dic[tag]['color'] = cmap.colors[i]
    tag_pos_dic[tag] = (tag_cir_dic[tag][0],tag_cir_dic[tag][1])
    tag_node_dic[tag]['radius'] = tag_cir_dic[tag][2]
    tag_node_dic[tag]['avg_size'] = sum(size_l)/len(size_l)
    i += 1

  for tag in tag_cir_dic:
    for tag2 in tag_cir_dic:
      if (tag != tag2) and ((tag2,tag) not in tag_edge_dic.keys()):
        sentiment_l = []
        width_l = []
        for edge in edge_dic:
          if (((node_dic[edge[0]]['tag'] == tag) and (node_dic[edge[1]]['tag'] ==  tag2)) or ((node_dic[edge[1]]['tag'] == tag) and (node_dic[edge[0]]['tag'] ==  tag2))):
            sentiment_l.append(edge_dic[edge]['sentiment'])
            width_l.append(edge_dic[edge]['width'])
        if sentiment_l and width_l:
          tag_edge_dic[(tag,tag2)] = {}
          tag_edge_dic[(tag,tag2)]['width'] = round(sum(width_l) / len(width_l),2)
          tag_edge_dic[(tag,tag2)]['sentiment'] = round(sum(sentiment_l) / len(sentiment_l),2)
  #tag_edge_dic = {z: v for z, v in tag_edge_dic.items() if v}
  G_all = nx.Graph()
  G_all.add_nodes_from(tag_node_dic)
  G_all.add_edges_from(tag_edge_dic)

  for edge in G_all.edges:
    if edge not in tag_edge_dic:
      tag_edge_dic[edge] = {}
      tag_edge_dic[edge]['width'] = tag_edge_dic[(edge[1],edge[0])]['width']
      tag_edge_dic[edge]['sentiment'] = tag_edge_dic[(edge[1],edge[0])]['sentiment']
      del tag_edge_dic[(edge[1],edge[0])]

  return G_all,tag_node_dic,tag_edge_dic,tag_pos_dic
G_all,tag_node_dic,tag_edge_dic,tag_pos_dic = overall_graph(n_dic,e_dic,tag_cir_dic)

## 7e. Overall Clustering of Sentential Co-Occurences by Term Attribute

In [0]:
def get_att_assort(G,n_dic,att='tag'):
  Gf=nx.Graph()
  i = 0
  att_list = []
  for node in G.nodes:
    att_list.append(n_dic[node][att])
  att_list =  set(att_list)
  for x in att_list:
    node_list = []
    for node in G.nodes:
      if x == n_dic[node][att]:
        node_list.append(node)
    Gf.add_nodes_from(node_list,att=n_dic[node_list[0]][att])
    i += 1
  Gf.add_edges_from(G.edges)
  return round(nx.attribute_assortativity_coefficient(Gf,'att'),2)

In [0]:
import matplotlib
plt.figure(figsize=(15,15))

edge_scalar = 20
node_scalar = 30000

sentiments = [e_dic[i]['sentiment'] for i in e_dic]
node_size = [tag_node_dic[i]['radius']*node_scalar for i in G_all.nodes]
colors = [tag_node_dic[i]['color'] for i in G_all.nodes]

tag_sentiment = [tag_edge_dic[i]['sentiment'] for i in G_all.edges]
widths = [math.log(tag_edge_dic[i]['width']+1)*edge_scalar for i in G_all.edges]

cmap = plt.cm.viridis_r
cmaplist = [cmap(i) for i in range(cmap.N)]
cmaplist[0] = (1.0,1.0,1.0,1.0)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent, vmax=max_sent)

sentiment_colors = [cmap(norm(i)) for i in tag_sentiment]

nx.draw(G_all,tag_pos_dic, with_labels=True, font_size = 18, font_weight = 'bold',alpha=1,node_size=node_size,node_color=colors)

nx.draw_networkx_edges(G_all,tag_pos_dic,edgelist=G_all.edges,width=widths,edge_color=sentiment_colors, alpha=1)

plt.title('Sentential Co-Occurrence by\nNLP Attribute Summarization',size=40)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm) #, orientation='horizontal')
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)
plt.xlim(-12.5,12.5)
plt.ylim(-15.2,13.7)

## 7f. Sentential Co-Occurence Clustering by Term Attribute

In [0]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import AxesGrid

plt.figure(figsize=(20,20))

edge_scalar = 4
node_scalar = 200

sentiments = [e_dic[i]['sentiment'] for i in G.edges]
node_size = [n_dic[i]['size']*node_scalar for i in G.nodes]
colors = [n_dic[i]['fluid_color'] for i in G.nodes]

sentiment = [e_dic[i]['sentiment'] for i in G.edges]
widths = [math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges]

cmap = plt.cm.viridis_r
cmaplist = [cmap(i) for i in range(cmap.N)]
cmaplist[0] = (1.0,1.0,1.0,1.0)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent*1.01, vmax=max_sent*1.01)

sentiment_colors = [cmap(norm(i)) for i in sentiment]

nx.draw(G, final_pos_dic, with_labels=True, font_size = 18, font_weight = 'bold',node_size = [n_dic[i]['size']*node_scalar for i in n_dic], node_color = colors, width = widths)

nx.draw_networkx_edges(G, final_pos_dic,edgelist=G.edges,width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges], alpha=1, edge_color=sentiment_colors)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

plt.title('Sentential Co-Occurrences by\nFluid Community (Color), Assort = {}\n NLP Object (Cluster), Assort = {}'.format(get_att_assort(G,n_dic,att='fluid_color'),get_att_assort(G,n_dic)),size=40)

In [0]:
import community
import operator

partition_lo = community.best_partition(G)

kk=len(set(partition_lo.values()))
modularity_lo=community.modularity(partition_lo, G, weight='weight')
print('The graph has', kk, 'Louvain communities and modularity coefficient equal to', modularity_lo)

cml=[]
for j in range(kk):
    cj=[]
    for k,v in partition_lo.items():
        if v==j:
            cj.append(k)
    cml.append(cj)
cmd={}
for j in range(kk):
    cmd[tuple(cml[j])]=len(cml[j])
cmd
ocmd=sorted(cmd.items(), key=operator.itemgetter(1), reverse=True)
eocmd=enumerate(ocmd)
commsd_lo={}
louvain_comm_membership={}
for i in eocmd:
    print('Community', i[0], 'with', i[1][1], 'nodes:', list(i[1][0]))
    commsd_lo[i[0]]=i[1][1] 
    ts=list(i[1][0])
    louvain_comm_membership[i[0]]=ts 

In [0]:
g = ig.Graph(directed=False)
g.add_vertices(list(G.nodes))
g.add_edges(G.edges)

partition_le = leidenalg.find_partition(g, leidenalg.ModularityVertexPartition);

partition_nodes = []
for part_list in partition_le:
  part = [list(G.nodes())[i] for i in part_list]
  partition_nodes.append(part)

cml=list(partition_nodes)
leiden_partition={}
for i,j in list(enumerate(cml)):
  for jj in j:
    leiden_partition[jj]=i
modularity_le=community.modularity(leiden_partition, G, weight='weight')
print('The graph has', kk, 'Leiden communities and modularity coefficient equal to', modularity_le)

cmd={}
kk=len(cml)
for j in range(kk):
  cmd[tuple(cml[j])]=len(cml[j])
cmd

ocmd=sorted(cmd.items(), key=operator.itemgetter(1), reverse=True)
eocmd=enumerate(ocmd)
commsd_le={}
leiden_comm_membership={}
for i in eocmd:
    print('Community', i[0], 'with', i[1][1], 'nodes:', list(i[1][0]))
    commsd_le[i[0]]=i[1][1] 
    ts=list(i[1][0])
    leiden_comm_membership[i[0]]=ts 

In [0]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import AxesGrid

for z,v in louvain_comm_membership.items():
  for node in v:
    n_dic[node]['louvain_partition'] = 'Louvain_Comm_' + str(z)

plt.figure(figsize=(20,20))

cmap = cm.get_cmap('Set3')

edge_scalar = 3
node_scalar = 300

communities = list(set([n_dic[i]['louvain_partition'] for i in G.nodes]))
sentiments = [e_dic[i]['sentiment'] for i in G.edges]
node_size = [n_dic[i]['size']*node_scalar for i in G.nodes]
colors = [cmap.colors[communities.index(n_dic[i]['louvain_partition'])] for i in G.nodes]

sentiment = [e_dic[i]['sentiment'] for i in G.edges]
widths = [math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges]

cmap = cm.get_cmap('viridis_r')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent*1.01, vmax=max_sent*1.01)

#G2,pos,sizes_G2,node_colors_G2 = add_center_circles(xychange,circle_list,tags_that_exist)

sentiment_colors = [cmap(norm(i)) for i in sentiment]

nx.draw(G, final_pos_dic, with_labels=True, font_size = 18, font_weight = 'bold',node_size = [n_dic[i]['size']*node_scalar for i in G.nodes], node_color = colors, width = widths)

nx.draw_networkx_edges(G, final_pos_dic,edgelist=G.edges,width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in e_dic], alpha=1, edge_color=sentiment_colors)
     
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

plt.title('Sentential Co-Occurrences by\nLouvain Community (Color), Assort = {}\n NLP Object (Cluster), Assort = {}'.format(get_att_assort(G,n_dic,'louvain_partition'),get_att_assort(G,n_dic)),size=40)

## 7g. Overall Clustering of Sentential Co-Occurences by Community

In [0]:
louvain_colors = []

cmap = cm.get_cmap('Set3')

for node in G.nodes:
  i = 0
  while i < len(louvain_comm_membership):
    if node in louvain_comm_membership[i]:
      n_dic[node]['louvain_color'] = cmap.colors[i]
    i += 1

k = 1.3
pos_part2 = nx.spring_layout(G,k)
pos_part2

node_colors = louvain_colors
coord_part2 = make_coordinates(len(louvain_comm_membership))
colors_that_exist = set(n_dic[i]['louvain_color'] for i in n_dic)
colors_that_exist

color_pos = []
for color in colors_that_exist:
  node_list2 = {}
  for node in G.nodes:
    if n_dic[node]['louvain_color'] == color:
      node_list2[node] = pos_part2[node]
  color_pos.append(node_list2)
color_pos

In [0]:
def circular_color_position(x_list,coord_list,pos,pos_list,n_dic):
  cir_pos_list = []
  #k = 0.8
  i = 0
  radius_scalar = 0.25 #2.5/coord_scale
  for l in x_list:
    G1 = nx.Graph()
    new_node_list = []
    for node in n_dic:
      if n_dic[node]['louvain_color'] == l:
        new_node_list.append(node)
    G1.add_nodes_from(new_node_list)
    #pos = nx.spring_layout(G1, k, iterations=17)
    pos = nx.circular_layout(G1,scale=len(G1.nodes)*radius_scalar)
    #print(pos)
    for key in pos.keys():
      pos[key] = [pos[key][0]+coord_list[i][0],pos[key][1]+coord_list[i][1]]
    cir_pos_list.append(pos)
    i += 1
  return cir_pos_list

cir_pos_list2 = circular_color_position(colors_that_exist,coord_part2,pos_part2,color_pos,n_dic)
circle_list2,rmax2 = show_circle_layout(cir_pos_list2,0.01)
xychange2 = find_xydelta(circle_list2,rmax2,3)
final_pos_dic2 = centralize_circles(cir_pos_list2,xychange2)

In [0]:
def color_by_tag(tags_that_exist,node_dic):
  i = 0
  cmap = cm.get_cmap('Set3')
  node_colors2 = []
  for node in node_dic:
    i = 0
    for tag in tags_that_exist:
      if node_dic[node]['tag'] == tag:
        node_colors2.append(cmap.colors[i])
      i += 1
  return node_colors2
node_colors2 = color_by_tag(tags_that_exist,n_dic)
node_colors2

In [0]:
louvain_pos = {}
i = 0
for v in sorted(set(louvain_comm_membership)):
  key = 'Louvain_Comm_'+ str(v)
  louvain_pos[key] = circle_list2[i]
  i += 1
louvain_pos

In [0]:
def overall_louvain_graph(node_dic,edge_dic,louvain_pos,node_scalar=10000,edge_scalar=20):
  cmap = cm.get_cmap('Set3')
  tag_node_dic = {}
  tag_edge_dic = {}
  tag_pos_dic = {}
  i = 0
  for tag in louvain_pos:
    node_num = 0
    tag_node_dic[tag] = {}
    size_l = []
    for node in node_dic:
      if node_dic[node]['louvain_partition'] == tag:
        node_num += 1
    tag_node_dic[tag]['node_num'] = node_num
    tag_node_dic[tag]['color'] = cmap.colors[i]
    tag_pos_dic[tag] = (louvain_pos[tag][0],louvain_pos[tag][1])
    tag_node_dic[tag]['radius'] = louvain_pos[tag][2]
    i += 1

  for tag in louvain_pos:
    for tag2 in louvain_pos:
      if (tag != tag2) and ((tag2,tag) not in tag_edge_dic.keys()):
        sentiment_l = []
        width_l = []
        for edge in edge_dic:
          if (((node_dic[edge[0]]['louvain_partition'] == tag) and (node_dic[edge[1]]['louvain_partition'] ==  tag2)) or ((node_dic[edge[1]]['louvain_partition'] == tag) and (node_dic[edge[0]]['louvain_partition'] ==  tag2))):
            sentiment_l.append(edge_dic[edge]['sentiment'])
            width_l.append(edge_dic[edge]['width'])
        if sentiment_l and width_l:
          tag_edge_dic[(tag,tag2)] = {}
          tag_edge_dic[(tag,tag2)]['width'] = round(sum(width_l) / len(width_l),2)
          tag_edge_dic[(tag,tag2)]['sentiment'] = round(sum(sentiment_l) / len(sentiment_l),2)
  #tag_edge_dic = {z: v for z, v in tag_edge_dic.items() if v}
  G_all = nx.Graph()
  G_all.add_nodes_from(tag_node_dic)
  G_all.add_edges_from(tag_edge_dic)

  for edge in G_all.edges:
    if edge not in tag_edge_dic:
      tag_edge_dic[edge] = {}
      tag_edge_dic[edge]['width'] = tag_edge_dic[(edge[1],edge[0])]['width']
      tag_edge_dic[edge]['sentiment'] = tag_edge_dic[(edge[1],edge[0])]['sentiment']
      del tag_edge_dic[(edge[1],edge[0])]

  return G_all,tag_node_dic,tag_edge_dic,tag_pos_dic
G_all,tag_node_dic,tag_edge_dic,tag_pos_dic = overall_louvain_graph(n_dic,e_dic,louvain_pos)

In [0]:
from matplotlib.pyplot import cm
plt.figure(figsize=(15,15))

edge_scalar = 14
node_scalar = 20000

plt.style.use('seaborn-deep')

sentiments = [tag_edge_dic[i]['sentiment'] for i in G_all.edges]
node_size = [tag_node_dic[i]['radius']*node_scalar for i in G_all.nodes]
colors = [tag_node_dic[i]['color'] for i in G_all.nodes]

tag_sentiment = [tag_edge_dic[i]['sentiment'] for i in G_all.edges]
widths = [math.log(tag_edge_dic[i]['width']+1)*edge_scalar for i in G_all.edges]

cmap = cm.get_cmap('viridis_r')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent*1.01, vmax=max_sent*1.01)

sentiment_colors = [cmap(norm(i)) for i in tag_sentiment]

nx.draw(G_all,tag_pos_dic, with_labels=True, font_size = 18, font_weight = 'bold',alpha=1,node_size=node_size,node_color=colors)

nx.draw_networkx_edges(G_all,tag_pos_dic,edgelist=G_all.edges,width=widths,edge_color=sentiment_colors, alpha=1)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

plt.title('Sentential Co-Occurrence by\nLouvain Community Summarization (1944)',size=40)
plt.xlim(-11.5,12.5)
plt.ylim(-15.2,13.7)

In [0]:
for z,v in louvain_comm_membership.items():
   print( 'Louvain Community ' + str(z) + ' contains ' + str(v))

In [0]:
for node in G.nodes:
  print(node,n_dic[node])

##7h. Sentential Co-Occurence Clustering by Community 

In [0]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.pyplot import cm

#G,n_dic,e_dic = make_G(df,'sentences',8,node_dic,sent_dic,sentiments2,'','Anne')

plt.figure(figsize=(20,15))

k = 0.8
edge_scalar = 2.5
node_scalar = 200

cmap = cm.get_cmap('viridis_r')
cmap2 = cm.get_cmap('Set3')

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

widths = [math.log(e_dic[i]['width']+1)*edge_scalar for i in G.edges]
sentiments = [e_dic[i]['sentiment'] for i in G.edges]
sizes = [n_dic[i]['size']*node_scalar for i in G.nodes]
node_color = [cmap2.colors[tags_that_exist.index(n_dic[i]['tag'])] for i in G.nodes]

nx.draw(G, final_pos_dic2, with_labels=True, font_size = 18, font_weight = 'bold',node_size = sizes, node_color = node_color, width = widths)

nx.draw_networkx_edges(G, final_pos_dic2,edgelist=G.edges,width=widths, alpha=1, edge_color=sentiments, edge_cmap=cmap)

plt.title('Sentential Co-Occurrences in 1944 by\nNLP Object (Color), Assort = {}\n Louvain Community (Cluster), Assort = {}'.format(get_att_assort(G,n_dic),get_att_assort(G,n_dic,'louvain_partition')),size=40)

##7i. Egocentric Graph of Diary/Sentential Co-Occurent Terms (Annex/Kitty/Anne)

In [0]:
plt.subplots(figsize=(15,15))

node_scalar = 250
edge_scalar = 3

ego_name = 'Kitty'

Gg,n_dic,e_dic = make_G(df,'sentences',0,node_dic,sent_dic,sentiments2,ego_name,'')

#pos = graphviz_layout(G)

sentiments = [e_dic[i]['sentiment'] for i in Gg.edges]

if min(sentiments) >= 0:
  min_sent = -1
else:
  min_sent = min(sentiments)

if max(sentiments) <= 0:
  max_sent = 1
else:
  max_sent = max(sentiments)

cmap = cm.get_cmap('viridis_r')
cmap2 = cm.get_cmap('Set3')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent*1.01, vmax=max_sent*1.01)

sentiment_colors = [cmap(norm(i)) for i in sentiments]

color = []
i = 0
for node in Gg.nodes:
  if node == ego_name:
    color.append(cmap2.colors[i])
  else:
    color.append(cmap2.colors[i+1])

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

node_size = []
for node in Gg.nodes:
  if node != ego_name:
    node_size.append(n_dic[node]['size']*node_scalar)
  else:
    node_size.append(n_dic[node]['size']*node_scalar*1.2)

node_and_degree = Gg.degree()
(largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
# Create ego graph of main hub
hub_ego = nx.ego_graph(Gg, largest_hub)
# Draw graphu
pos = nx.spring_layout(hub_ego)
#hub_ego.remove_node(ego_name)
nx.draw(hub_ego, pos, node_color=cmap2.colors[4], node_size=node_size, with_labels=True,font_size=12,font_weight='bold',width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in Gg.edges],edge_color=sentiment_colors)
# Draw ego as large and red
nx.draw_networkx_nodes(hub_ego.remove_node(ego_name), pos, nodelist=[largest_hub], node_size=n_dic[ego_name]['size']*node_scalar, node_color=cmap2.colors[3], with_labels=True,font_size=12,font_weight='bold')

plt.title('Egocentric Sentential Co-Occurrence of {}'.format(ego_name),size=40)

In [0]:
plt.subplots(figsize=(15,15))

node_scalar = 250
edge_scalar = 3

ego_name = 'Anne'

Gg,n_dic,e_dic = make_G(df,'sentences',6,node_dic,sent_dic,sentiments2,ego_name,'')

#pos = graphviz_layout(G)

sentiments = [e_dic[i]['sentiment'] for i in Gg.edges]

if min(sentiments) >= 0:
  min_sent = -1
else:
  min_sent = min(sentiments)

if max(sentiments) <= 0:
  max_sent = 1
else:
  max_sent = max(sentiments)

cmap = cm.get_cmap('viridis_r')
cmap2 = cm.get_cmap('Set3')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent*1.01, vmax=max_sent*1.01)

sentiment_colors = [cmap(norm(i)) for i in sentiments]

color = []
i = 0
for node in Gg.nodes:
  if node == ego_name:
    color.append(cmap2.colors[i])
  else:
    color.append(cmap2.colors[i+1])

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

node_size = []
for node in Gg.nodes:
  if node != ego_name:
    node_size.append(n_dic[node]['size']*node_scalar)
  else:
    node_size.append(n_dic[node]['size']*node_scalar*1.2)

node_and_degree = Gg.degree()
(largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
# Create ego graph of main hub
hub_ego = nx.ego_graph(Gg, largest_hub)
# Draw graphu
pos = nx.spring_layout(hub_ego,3)
#hub_ego.remove_node(ego_name)
nx.draw(hub_ego, pos, node_color=cmap2.colors[4], node_size=node_size, with_labels=True,font_size=12,font_weight='bold',width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in Gg.edges],edge_color=sentiment_colors)
# Draw ego as large and red
nx.draw_networkx_nodes(hub_ego.remove_node(ego_name), pos, nodelist=[largest_hub], node_size=n_dic[ego_name]['size']*node_scalar, node_color=cmap2.colors[3], with_labels=True,font_size=12,font_weight='bold')

plt.title('Egocentric Sentential Co-Occurrence of {}'.format(ego_name),size=40)

In [0]:
plt.subplots(figsize=(15,15))

node_scalar = 250
edge_scalar = 3

ego_name = 'Annex'

Gg,n_dic,e_dic = make_G(df,'sentences',8,node_dic,sent_dic,sentiments2,ego_name,'')

#pos = graphviz_layout(G)

sentiments = [e_dic[i]['sentiment'] for i in Gg.edges]

if min(sentiments) >= 0:
  min_sent = -1
else:
  min_sent = min(sentiments)

if max(sentiments) <= 0:
  max_sent = 1
else:
  max_sent = max(sentiments)

cmap = cm.get_cmap('viridis_r')
cmap2 = cm.get_cmap('Set3')

norm = matplotlib.colors.DivergingNorm(vcenter=0, vmin=min_sent*1.01, vmax=max_sent*1.01)

sentiment_colors = [cmap(norm(i)) for i in sentiments]

color = []
i = 0
for node in Gg.nodes:
  if node == ego_name:
    color.append(cmap2.colors[i])
  else:
    color.append(cmap2.colors[i+1])

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_sent, vmax=max_sent))
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.ax.set_ylabel('Sentiment',size=30,rotation=270,labelpad=20)

node_size = []
for node in Gg.nodes:
  if node != ego_name:
    node_size.append(n_dic[node]['size']*node_scalar)
  else:
    node_size.append(n_dic[node]['size']*node_scalar*1.2)

node_and_degree = Gg.degree()
(largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
# Create ego graph of main hub
hub_ego = nx.ego_graph(Gg, largest_hub)
# Draw graphu
pos = nx.spring_layout(hub_ego,2)
#hub_ego.remove_node(ego_name)
nx.draw(hub_ego, pos, node_color=cmap2.colors[4], node_size=node_size, with_labels=True,font_size=12,font_weight='bold',width=[math.log(e_dic[i]['width']+1)*edge_scalar for i in Gg.edges],edge_color=sentiment_colors)
# Draw ego as large and red
nx.draw_networkx_nodes(hub_ego.remove_node(ego_name), pos, nodelist=[largest_hub], node_size=n_dic[ego_name]['size']*node_scalar, node_color=cmap2.colors[3], with_labels=True,font_size=12,font_weight='bold')

plt.title('Egocentric Sentential Co-Occurrence of {}'.format(ego_name),size=40)

##8. Graph Partitions

###8a. Partition of Fluid Communities

In [0]:
commsd_fl = {}
for z,v in color_comm_membership.items():
  commsd_fl[z] = len(v)
ddf=pd.DataFrame(commsd_fl.items(), columns=['Community', 'Number of Nodes'])
sst="Bar plot of Fluid Communities"
ddf.plot.bar(x='Community', y='Number of Nodes', figsize=(12,7),rot=0,title=sst);

In [0]:
#G,n_dic,e_dic = make_G(df,'sentences',14,node_dic,sent_dic,sentiments2,'','Anne')

node_color = []
for node in G.nodes:
  for z,v in color_comm_membership.items():
    if node in v:
      node_color.append(z)

vmin = min(node_color) #df['color'].min()
vmax = max(node_color) #df['color'].max()
cmap=plt.cm.viridis #plt.cm.coolwarm #plt.cm.Blues #

colors = [i for i in node_color]

plt.figure(figsize=(10,7));
node_border_color='k'
nodes = nx.draw_networkx_nodes(G, final_pos_dic, node_color=colors, cmap=cmap, vmin=vmin, vmax=vmax)
nodes.set_edgecolor(node_border_color)
nx.draw_networkx_edges(G, final_pos_dic,edge_color='b',alpha=0.8)
plt.axis('off');
yoffset = {}
y_off = -0.7 #0.05  # offset on the y axis
for z, v in final_pos_dic.items():
    yoffset[z] = (v[0], v[1]+y_off)
nx.draw_networkx_labels(G, yoffset,font_weight='bold',font_size='11');
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
cbar = plt.colorbar(sm,label="Fluid Communities") #, orientation='horizontal')
sst="Fluid Communities Clustered by NLP Attribute"
plt.title(sst,fontsize=15);
plt.margins(x=0.1, y=0.1)

###8b. Partition of Lovuain Communities

In [0]:
ddf=pd.DataFrame(commsd_lo.items(), columns=['Community', 'Number of Nodes'])
sst="Bar plot of Louvain Communities"
ddf.plot.bar(x='Community', y='Number of Nodes', figsize=(12,7),rot=0,title=sst);

In [0]:
#G,n_dic,e_dic = make_G(df,'sentences',14,node_dic,sent_dic,sentiments2,'','Anne')

partition_lo = community.best_partition(G)
node_color=partition_lo.values()
vmin = min(node_color) #df['color'].min()
vmax = max(node_color) #df['color'].max()
cmap=plt.cm.viridis #plt.cm.coolwarm #plt.cm.Blues #

colors = [i for i in node_color]

plt.figure(figsize=(10,7));
node_border_color='k'
nodes = nx.draw_networkx_nodes(G, final_pos_dic, node_color=colors, cmap=cmap, vmin=vmin, vmax=vmax)
nodes.set_edgecolor(node_border_color)
nx.draw_networkx_edges(G, final_pos_dic,edge_color='b',alpha=0.8)
plt.axis('off');
yoffset = {}
y_off = -0.7 #0.05  # offset on the y axis
for z, v in final_pos_dic.items():
    yoffset[z] = (v[0], v[1]+y_off)
nx.draw_networkx_labels(G, yoffset,font_weight='bold',font_size='11');
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
cbar = plt.colorbar(sm,label="Louvain Communities") #, orientation='horizontal')
sst="Louvain Communities Clustered by NLP Attribute"
plt.title(sst,fontsize=15);
plt.margins(x=0.1, y=0.1)

###8c. Partition of Leiden Communities

In [0]:
ddf=pd.DataFrame(commsd_le.items(), columns=['Community', 'Number of Nodes'])
sst="Bar plot of Leiden Communities"
ddf.plot.bar(x='Community', y='Number of Nodes', figsize=(12,7),rot=0,title=sst);

In [0]:
#G,n_dic,e_dic = make_G(df,'sentences',14,node_dic,sent_dic,sentiments2,'','Anne')

node_color = []
for node in G.nodes:
  for z,v in leiden_comm_membership.items():
    if node in v:
      node_color.append(z)

vmin = min(node_color) #df['color'].min()
vmax = max(node_color) #df['color'].max()
cmap=plt.cm.viridis #plt.cm.coolwarm #plt.cm.Blues #

for part_list in partition_le:
  part = [list(G.nodes())[i] for i in part_list]
  partition_nodes.append(part)

colors = [i for i in node_color]

plt.figure(figsize=(10,7));
node_border_color='k'
nodes = nx.draw_networkx_nodes(G, final_pos_dic, node_color=colors, cmap=cmap, vmin=vmin, vmax=vmax)
nodes.set_edgecolor(node_border_color)
nx.draw_networkx_edges(G, final_pos_dic,edge_color='b',alpha=0.8)
plt.axis('off');
yoffset = {}
y_off = -0.7 #0.05  # offset on the y axis
for z, v in final_pos_dic.items():
    yoffset[z] = (v[0], v[1]+y_off)
nx.draw_networkx_labels(G, yoffset,font_weight='bold',font_size='11');
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
cbar = plt.colorbar(sm,label="Leiden Communities") #, orientation='horizontal')
sst="Leiden Communities Clustered by NLP Attribute"
plt.title(sst,fontsize=15);
plt.margins(x=0.1, y=0.1)

###8d. Community Symmetric Differences

In [0]:
def ccomp(A,type1,B,type2):
    eds=[]
    A1=set(A.keys())
    B1=set(B.keys())
    for i in A1:
        x=set(A[i])
        for j in B1:
            y=set(B[j])
            sd=sorted(list(x.symmetric_difference(y)))
            ss=sorted(list(set(list(x)+list(y))))
            if sorted(A[i])==sorted(B[j]):
                eds.append((type1+str(i),type2+str(j),1.))
            else:
                if len(sd)/float(len(ss))==1:
                    eds.append((type1+str(i),type2+str(j),0.))
                else:
                    eds.append((type1+str(i),type2+str(j),len(sd)/float(len(ss))))
    H = nx.Graph()
    H.add_weighted_edges_from(eds)
    H.remove_edges_from([e for e in H.edges(data=True) if e[2]['weight']==0.])       
    return H

In [0]:
fluid_partition = {}
for k,v in color_comm_membership.items():
  for x in v:
    fluid_partition[x] = k

type1='Louvain_Comm_'
type2='Leiden_Comm_'
type3='Fluid_Comm_'
H=ccomp(louvain_comm_membership,type1,leiden_comm_membership,type2)
H1=ccomp(louvain_comm_membership,type1,color_comm_membership,type3)
H2=ccomp(leiden_comm_membership,type2,color_comm_membership,type3)
H.add_nodes_from(H1.nodes)
H.add_nodes_from(H2.nodes)
H.add_weighted_edges_from([(i[0],i[1],H1.edges[i]['weight']) for i in H1.edges])
H.add_weighted_edges_from([(i[0],i[1],H2.edges[i]['weight']) for i in H2.edges])

edge_scalar = 10

posbp={}
louvain=sorted([n for n in H.nodes() if "Louvain" in n])
leiden=sorted([n for n in H.nodes() if "Leiden" in n])
fluid=sorted([n for n in H.nodes() if "Fluid" in n])

i=0
for node in louvain:
  xs = np.linspace(1.0, 5.0, num=len(louvain))
  posbp[node] = (xs[i],0)
  i += 1

i=0
for node in leiden:
  xs = np.linspace(0.5, 2, num=len(leiden))
  posbp[node] = (xs[i],-0.57*xs[i])
  i += 1

i=0
for node in fluid:
  xs = np.linspace(4, 6.0, num=len(fluid))
  posbp[node] = (xs[i],0.57*xs[i]-3.7)
  i += 1

elabels={}
elabels = nx.get_edge_attributes(H,'weight')
elabelsc={}
for z,v in elabels.items():
    elabelsc[z]='%.02f' %v 
elabels=elabelsc

plt.figure(figsize=(12,10));
widths = [float(i) for i in list(elabels.values())]
widths = [i/max(widths)*edge_scalar for i in widths]
nx.draw(H, posbp, edgelist=H.edges(), edge_color=widths, width=10.0, edge_cmap=plt.cm.Blues)
nodes1 = nx.draw_networkx_nodes(H, posbp, nodelist=louvain,node_color="b",node_shape='s',alpha=0.4)
nodes2 = nx.draw_networkx_nodes(H, posbp, nodelist=leiden,node_color="g",node_shape='s',alpha=0.4)
nodes3 = nx.draw_networkx_nodes(H, posbp, nodelist=fluid,node_color="r",node_shape='s',alpha=0.4)
#nx.draw_networkx_edges(H, posbp, edge_colors=widths,width=4, alpha=0.7,edge_cmap=plt.cm.Blues)
# nx.draw_networkx_labels(H, posbp)
#nx.draw_networkx_edge_labels(H,posbp,edge_labels=elabels,label_pos = 0.7,font_size=7);
plt.axis('off');
yoffset = {}
y_off = -0.05  # offset on the y axis
for z, v in posbp.items():
    yoffset[z] = (v[0], v[1]+y_off)
nx.draw_networkx_labels(H, yoffset,font_weight='bold');
plt.margins(x=0.1, y=0.1)
plt.title("Similarities-Weighted Communities\nfor the Multilayer Graph (1944)",size=40);

###8f. Modularity Comparison by Community

In [0]:
modularity_fl=community.modularity(fluid_partition, G, weight='weight')

In [0]:
import networkx.algorithms.community.quality as qu
loucomms=[v for k,v in louvain_comm_membership.items()]
leicomms=[v for k,v in leiden_comm_membership.items()]
fluidcomms=[v for k,v in color_comm_membership.items()]
perf_lo=qu.performance(G,loucomms)
perf_le=qu.performance(G,leicomms)
perf_fl=qu.performance(G,fluidcomms)

In [0]:
cov_lo=qu.coverage(G,loucomms)
cov_le=qu.coverage(G,leicomms)
cov_fl=qu.coverage(G,fluidcomms)

In [0]:
plt.style.use('seaborn-dark')

labels = ['Louvain','Leiden','Fluid']
modularities = [modularity_lo,modularity_le,modularity_fl]
performances = [perf_lo,perf_le,perf_fl]
coverage = [cov_lo,cov_le,cov_fl]

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars
buf = 0.1

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, modularities, width, label='Modularity')
rects2 = ax.bar(x, performances, width, label='Performance')
rects3 = ax.bar(x + width, coverage, width, label='Coverage')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Metric Score',size=18)
ax.set_title('Community Metrics',size=24)
ax.set_xticks(x)
ax.set_xticklabels(labels,size=18)
ax.legend()

fig.tight_layout()

fig.set_figheight(10)
fig.set_figwidth(8)
plt.axhline(y=0, color='k', linestyle='-')
plt.show()

##9. Cliques

###9a. Clique Identification

In [0]:
G_cl,n_dic_cl,e_dic_cl = make_G(df,'sentences',10,node_dic,sent_dic,sentiments2,'','')

cliques=[clique for clique in nx.find_cliques(G_cl)]
gcliques=[clique for clique in cliques if len(clique)>2]
lcliques=[clique for clique in cliques if len(clique)<=2]
print("The total number of cliques is %i" %len(cliques))
print("The total number of non trivial cliques is %i" %len(gcliques))
print("The total number of trivial cliques is %i" %len(lcliques))

###9b. Bipartite Clique Graph

In [0]:
n2cdi={}
for n in G_cl.nodes():
    for w in gcliques:
        if n in w:
            n2cdi[n]=w

cdi={}
for i,j in list(enumerate(cliques)):
    if i+1<10:
        cdi["clique_0"+str(i+1)]=j
    else:
        cdi["clique_"+str(i+1)]=j

gcdi={k:v for k,v in cdi.items() if len(v)>2}

eds=[]
for k,v in cdi.items(): #g
    for vv in v:
        eds.append((k,vv))
H=nx.Graph()
H.add_edges_from(eds)
Y=sorted([n for n in list(H.nodes()) if type(n)==str and "clique_" in n])
X=sorted([n for n in H.nodes() if n not in Y])

pu=1.

posbp={}
if len(X)==max([len(X),len(Y)]):
    for i,x in list(enumerate(X)):
        posbp[x]=(0,i)
    for j,y in list(enumerate(Y)):
        posbp[y]=(1,(j+j*float(i))/len(Y))
else:
    for i,x in list(enumerate(Y)):
        posbp[x]=(1,i)
    for j,y in list(enumerate(X)):
        posbp[y]=(0,(j+j*float(i))/len(Y))

plt.figure(figsize=(7,10));
nodes1 = nx.draw_networkx_nodes(H, pos=posbp,node_size=100,nodelist=Y,node_color="#b3ffb3",node_shape='s')
nodes2 = nx.draw_networkx_nodes(H, pos=posbp,node_size=100,nodelist=X,node_color="#ffb3b3",node_shape='o')

nx.draw_networkx_edges(H, pos=posbp, edge_color="b", alpha=0.5)
nx.draw_networkx_labels(H, pos=posbp,font_size=10)
plt.axis('off');

plt.margins(x=0.25, y=0.1)
sst="The bipartite graph of nodes vs. cliques"
plt.title(sst);

###9c. Clique Intersection

In [0]:
from itertools import combinations

cleds=[]
# sbeds=[]
for k,v in gcdi.items(): 
    ve=G_cl.subgraph(v).edges()
    for w in ve:
        cleds.append(w)
cleds+=[(c2,c1) for (c1,c2) in cleds]

sbeds=[e for e in G_cl.edges() if e not in cleds]


ceds=[]
for c in list(combinations(Y, 2)):
    ints=set(cdi[c[0]]).intersection(cdi[c[1]])
    if len(ints)>0:
        ceds.append((c[0],c[1],len(ints),list(ints)))

weight={(i,j):(k,l) for (i,j,k,l) in ceds}  
    
w_edges=[(x,y,z) for (x,y),z in weight.items()]

cG = nx.Graph()
cG.add_weighted_edges_from(w_edges)

elabels={}
elabels = nx.get_edge_attributes(cG,'weight')
edge_width=[cG[u][v]['weight'][0] for u,v in cG.edges()] 
edge_width=[w for w in edge_width]

lcdi={k:len(v) for k,v in cdi.items()}
nx.set_node_attributes(cG, lcdi, name="clique_size")

nodes_between_2cliques={}
tt=[]
for (x,y,z) in cG.edges(data=True):
    for xx in cdi[x]:
        if xx not in n2cdi.keys():
            for yy in cdi[y]:
                if yy not in n2cdi.keys():
                    tt.append((xx,yy,z))
for (x,y,z) in tt:
    if z['weight'][0]==1:
        if z['weight'][1][0] not in n2cdi.keys():
#         if lcdi[x]==2 and lcdi[y]==2:
            nodes_between_2cliques[(x,y)]=z['weight'][1][0]
# print len(set(nodes_between_2cliques.values())),list(set(nodes_between_2cliques.values()))

figsize=(15,15)
pos=graphviz_layout(cG); #pos=nx.circular_layout(G); 
node_color="#b3ffb3" #"#ffb3b3"
node_border_color="g"
edge_color="#668cff"
plt.figure(figsize=figsize);
nodes = nx.draw_networkx_nodes(cG, pos, node_size=[100*v for v in lcdi.values()],node_color=node_color,node_shape="s")
nodes.set_edgecolor(node_border_color)
nx.draw_networkx_edges(cG, pos, edge_color=edge_color,width=edge_width)
nx.draw_networkx_labels(cG, pos)
# nx.draw_networkx_edge_labels(cG,pos,edge_labels=elabels);
plt.axis('off');
sst="The intersection graph of cliques"
plt.title(sst);

###9d. Clustering of Clique Intersection

In [0]:
intersecting_clique_edges=[]
for e in cG.edges(data=True):
    a=cdi[e[0]]
    b=cdi[e[1]]
    c=set(a).intersection(set(b))
    if len(c)>1:
        sG=G_cl.subgraph(list(c))
        for se in sG.edges():
            if se not in intersecting_clique_edges:
                intersecting_clique_edges.append(se)

leaves=[n for n in G_cl.nodes() if G_cl.degree(n)==1]

pendants=[e for e in G_cl.edges() if e[0] in leaves or e[1] in leaves]

nb2c=list(set(nodes_between_2cliques.values()))

pos=graphviz_layout(G_cl,'neato')
plt.figure(figsize=(15,15));

nx.draw_networkx_edges(G_cl, pos, edgelist=[e for e in sbeds if e not in pendants],width=3,edge_color="r", alpha=0.7)
nx.draw_networkx_edges(G_cl, pos, edgelist=pendants,width=3,edge_color="orange", alpha=0.5)
nx.draw_networkx_edges(G_cl, pos, edgelist=[e for e in G_cl.edges() if e not in sbeds and e not in pendants and e not in intersecting_clique_edges],width=3,edge_color="c", alpha=0.7)
nx.draw_networkx_edges(G_cl, pos, edgelist=intersecting_clique_edges,width=3,edge_color="b", alpha=0.8)
nodes = nx.draw_networkx_nodes(G_cl, pos, nodelist=[n for n in G_cl.nodes() if n not in nb2c and n not in leaves],node_size=500,node_color="r", alpha=0.9)
nodes = nx.draw_networkx_nodes(G_cl, pos, nodelist=nb2c,node_size=500,node_color="g", alpha=0.9)
nodes = nx.draw_networkx_nodes(G_cl, pos, nodelist=leaves,node_size=500,node_color="orange", alpha=0.9)

plt.axis('off');
yoffset = {}
y_off = - 15
for k, v in pos.items():
    yoffset[k] = (v[0], v[1]+y_off)
nx.draw_networkx_labels(G_cl, yoffset,font_weight='bold',font_size=13);
plt.margins(x=0.25, y=0.2)
sst="Cliques of Sentential Co-Occurrence (1944)"#\n green nodes = brokers between cliques \n orange nodes = leaves \n red edges = bridges between cliques \n cyan edges = non-intersecting clique edges \n blue edges = intersecting clique edges"
plt.title(sst,size=35);

##10. Network Statistics

##10a. Centralities

In [0]:
def create_centralities_list(G_cl,maxiter=2000,pphi=5,centList=[]):
    if len(centList)==0:
        centList=['degree','closeness','betweenness','eigenvector','Katz','PageRank','HITS','load','communicability','current flow']
    cenLen=len(centList)
    valus={}
    for uu,centr in enumerate(centList):
        if centr=='degree':
            if isinstance(G_cl,nx.DiGraph):
                cent=nx.in_degree_centrality(G_cl)
                sstt='In Degree Centralities '
                valus['in_degree']=cent
                cent=nx.out_degree_centrality(G_cl)
                sstt+= 'and Out Degree Centralities'
                valus['out_degree']=cent
            else:
                cent=nx.degree_centrality(G_cl)
                sstt='Degree Centralities'
                ssttt='degree centrality'
                valus[centr]=cent
        elif centr=='closeness':
            cent=nx.closeness_centrality(G_cl)
            sstt='Closeness Centralities'
            ssttt='closeness centrality'
            valus[centr]=cent
        elif centr =='load':
            cent=nx.load_centrality(G_cl)
            sstt='Load Centraities'
            valus[centr]=cent
        elif centr == 'communicability':
            if not isinstance(G_cl, nx.DiGraph):
                cent=nx.communicability_betweenness_centrality(G_cl)
                sstt='Communicability Centralities'
                valus[centr]=cent
        elif centr=='betweenness':
            cent=nx.betweenness_centrality(G_cl)
            sstt='Betweenness Centralities'
            ssttt='betweenness centrality'
            valus[centr]=cent
        elif centr=='current flow':
            if not isinstance(G_cl, nx.DiGraph):
            
                cent=nx.current_flow_closeness_centrality(G_cl)
                sstt='Current Flow Closeness Centrality'
                valus[centr]=cent
        elif centr=='eigenvector':
            try:
                cent=nx.eigenvector_centrality(G_cl,max_iter=maxiter)
                sstt='Eigenvector Centralities'
                ssttt='eigenvector centrality'
                valus[centr]=cent

            except:
                valus[centr]=None

                continue
        elif centr=='Katz':
            phi = (1+math.sqrt(pphi))/2.0 # largest eigenvalue of adj matrix
            cent=nx.katz_centrality_numpy(G_cl,1/phi-0.01)
            cent=nx.katz_centrality_numpy(G_cl,.05)#,1/phi-0.01)
            
            sstt='Katz Centralities'
            ssttt='Katz centrality'
            valus[centr]=cent
#             valus[centr+'_%i' %pphi]=cent

        elif centr=='PageRank':
            try:
                cent=nx.pagerank(G_cl)
                sstt='PageRank'
                ssttt='pagerank'
                valus[centr]=cent

            except:
                valus[centr]=None

                continue
        elif centr=='HITS':
            if isinstance(G_cl,nx.DiGraph):
                dd=nx.hits(G_cl,max_iter=maxiter)
                sstt='HITS hubs '
                valus['HITS_hubs']=dd[0]
                sstt+= 'and HITS authorities'
                valus['HITS_auths']=dd[1]
            else:
                dd=nx.hits(G_cl,max_iter=maxiter)
                cent=nx.degree_centrality(G_cl)
                sstt='HITS'
                ssttt='HITS Centralities'
                valus[centr]=dd[0]
        else:
            continue
        print('%s done!!!' %sstt)
    return valus

dindices=['out_degree','in_degree','closeness','betweenness','eigenvector','HITS_hubs','HITS_auths','Katz','PageRank','load']
indices=['degree','closeness','betweenness','eigenvector','HITS','Katz','PageRank','load','communicability','current flow']
# indices=['degree','closeness','betweenness','eigenvector']

# Without 'communicability' and 'current flow' (undirected case)
dindicesd=['out_degree','in_degree','closeness','betweenness','eigenvector','HITS_hubs','HITS_auths','Katz','PageRank','load']
indicesd=['degree','closeness','betweenness','eigenvector','HITS','Katz','PageRank','load']
# indicesd=['degree','closeness','betweenness','eigenvector']

dindicesdr=dindices
indicesdr=indices

# Plus 'node'
dindicesdrn=["node"]+dindices
indicesdrn=['node']+indices

def picker(G_cl,data,cols,thres):
    for al in range(1,len(G_cl.nodes())+1):
        degs_dic={}
        deggs=None
        ndegs_dic={}
        for col in cols:
            if col not in ndegs_dic:
                ndegs_dic[col]={}
            degress=central_pd.sort_values(col,ascending=False).head(al).to_dict()
            if deggs is None:
                deggs=set(degress[col].keys())
            else:
                deggs=deggs.intersection(set(degress[col].keys()))
            degs_dic[col]=degress[col]#.keys()
            for ije, vvf in enumerate(sorted(set(degress[col].values()),reverse=True)):
                ndegs_dic[col][vvf]=ije+1
        if len(deggs)>=thres:
            break

    rank_dic=Counter()
    rrank_dic={}
    for i in deggs:
        ss='Key-player node %s is ranked: \n' %i  #has ranking index
        if i not in rrank_dic:
            rrank_dic[i]=[]
        for nn,kk in degs_dic.items():
            ss+='%i-th wrt %s (%f)\n' %(ndegs_dic[nn][kk[i]],nn,kk[i])#(kk.index(i)+1,nn)
            rank_dic[i]+=ndegs_dic[nn][kk[i]]
            rrank_dic[i].append(ndegs_dic[nn][kk[i]])
#         print ss
    return al,deggs,degs_dic,rank_dic,rrank_dic,ndegs_dic

In [0]:
central_pd=pd.DataFrame(create_centralities_list(G_cl))
central_pd["node"]=central_pd.index.values
if isinstance(G_cl,nx.DiGraph):
    central_pd=central_pd[['node']+dindices]
else:
    central_pd=central_pd[['node']+indices]
central_pd["node"]=central_pd.index.values
central_pd.reset_index(drop = True, inplace = True)
central_pd.sort_values('node') #.head()
central_pd

In [0]:
data = central_pd

if isinstance(G_cl,nx.DiGraph):
#     cols=dindicesdr# no hits
    cols=dindices
else:
#     cols=indicesdr#no hits
    cols=indices
    
thres=len(G_cl.nodes())

# al,deggs,degs_dic,rankings,rankd = picker(G,data,cols,thres) #,thres=1)
al,deggs,degs_dic,rankings,rrank_dic,ndegs_dic = picker(G_cl,data,cols,thres) #,thres=1)

central_pd=data.set_index('node')
nndegs_dic={}
ppdff=central_pd.to_dict(orient='dict')

for k,v in ppdff.items():
    nndegs_dic[k]=[ndegs_dic[k][j] for j in v.values()]
    
nndegs_dic['node']=[j.keys() for j in ppdff.values()][0]

fpdngegs=pd.DataFrame.from_dict(nndegs_dic, orient='index')
fpdngegs=fpdngegs.T
fpdngegs

In [0]:
if isinstance(G_cl,nx.DiGraph):
    df_c=fpdngegs[dindices]
else:
    df_c=fpdngegs[indices]
    
rd={}
for i in df_c.index.values:
    vv=df_c.iloc[i].tolist()
    ra=min(vv)
    vv=len(df_c.columns)*[ra]
    if isinstance(G_cl,nx.DiGraph):
        dff=df_c[(df_c['out_degree'] >= vv[0]) & (df_c['in_degree'] >= vv[1]) & (df_c['closeness'] >= vv[2]) & (df_c['betweenness'] >= vv[3]) & (df_c['eigenvector'] >= vv[4]) & (df_c['HITS_hubs'] >= vv[5]) & (df_c['HITS_auths'] >= vv[6]) & (df_c['Katz'] >= vv[7]) & (df_c['PageRank'] >= vv[8]) & (df_c['load'] >= vv[9])]   
    else:
        dff=df_c[(df_c['degree'] >= vv[0]) & (df_c['closeness'] >= vv[1]) & (df_c['betweenness'] >= vv[2]) & (df_c['eigenvector'] >= vv[3]) & (df_c['Katz'] >= vv[4]) & (df_c['PageRank'] >= vv[5]) & (df_c['load'] >= vv[6]) & (df_c['communicability'] >= vv[7]) & (df_c['current flow'] >= vv[8])]             
    rd[i]=len(dff)

In [0]:
mrd={}
for i,v in rd.items():
    mrd[fpdngegs.iloc[i]["node"]]=max(rd.values())-rd[i]+1
mrd

In [0]:
nn=[]
rn=[]
ru=[]
rankings=rd
en=enumerate(sorted(rankings, key=rankings.get, reverse=False))
r = {key: rank for rank, key in enumerate(sorted(set(rankings.values()), reverse=False), 1)}
rdd={k: r[v] for k,v in rankings.items()}
for i, w in en:
    nn.append(fpdngegs.iloc[w]['node'])  
    rn.append(max(rdd.values())-rdd[w]+1) 
    ru.append(rankings[w])
rdf=pd.DataFrame(
    {'node': nn, 
     'rung' : ru,
     'rank': rn
    })
rdf=rdf[["node","rung","rank"]]
rdf.sort_values("rank", inplace=True)
rdf

In [0]:
rdf[indices]=fpdngegs[indices]
import seaborn as sns
mrdf = rdf
cm = sns.light_palette("green", as_cmap=True)
mrdf[['degree','closeness','betweenness','eigenvector','HITS','Katz','PageRank','load','communicability','current flow']] = mrdf[['degree','closeness','betweenness','eigenvector','HITS','Katz','PageRank','load','communicability','current flow']].apply(pd.to_numeric)
mrdf.style.background_gradient(cmap='viridis')

In [0]:
clr={}
for i in range(len(rdf)):
    clr[rdf.iloc[i]["node"]]=rdf.iloc[i]["rank"]
node_color=[clr[n] for n in G_cl.nodes()]   

nsi=[50*G_cl.degree(n) for n in G_cl.nodes()]

labels={}
for i in G_cl.nodes():
    labels[i]=i

edge_scalar = 2
node_scalar = 3
nsi = [i*node_scalar for i in nsi]
edge_width = [math.log(e_dic_cl[i]['width']+1)*edge_scalar for i in G_cl.edges]

plt.figure(figsize=(18,10))
nx.draw_networkx_edges(G_cl,pos=pos,width=edge_width, edge_color="lime", alpha=0.8);
nc=nx.draw_networkx_nodes(G_cl,pos=pos,cmap=plt.get_cmap('rainbow'), node_color=node_color,edgecolors='r',node_size=nsi,alpha=1);
nx.draw_networkx_labels(G_cl,pos=pos,labels=labels,font_size=12);

cbar=plt.colorbar(nc)
#cbar.set_label('Aggregated Centralities Rating')
cbar.ax.set_ylabel('Aggregated Centralities Rating',size=20,rotation=270,labelpad=30)
plt.axis("off");
tt="Sentential Co-Occurrence\nNodes Colored According to Aggregated Centrality Rating"
plt.title(tt,size=25);

In [0]:
for l in sent_list:
  if ('van D.' in l) and ('Boche' in l):
    print(l)

In [0]:
name = 'Sentential Co-Occurrences'

df4 = df.loc[ list(G_cl.nodes) , list(G_cl.nodes) ].copy()#df3[list(node_dic.keys())].copy()
axes = pd.plotting.scatter_matrix(df4, figsize=(15, 15), marker='o',hist_kwds={'bins': 20}, s=60, alpha=.8)
corr = df4.corr().values 
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='left', va='center',rotation='vertical')
sst="%s \n Scatter Matrix of Centrality Indices \n (displaying Pearson correlation coefficients)" %name

n = len(G_cl.nodes)

for x in range(n):
    for y in range(n):
        # to get the axis of subplots
        ax = axes[x, y]
        # to make x axis name vertical  
        ax.xaxis.label.set_rotation(90)
        # to make y axis name horizontal 
        ax.yaxis.label.set_rotation(0)
        # to make sure y axis names are outside the plot area
        ax.yaxis.labelpad = 20

plt.suptitle(sst,fontsize=20);

In [0]:
name = 'Sentential Co-Occurrence'
pos=graphviz_layout(G_cl,'circo')
valus=create_centralities_list(G_cl,maxiter=2000,pphi=5,centList=[])
if isinstance(G_cl,nx.DiGraph):
    cts=['out_degree','in_degree','closeness','betweenness','eigenvector','HITS_hubs','HITS_auths','Katz','PageRank','load']
else:
    cts=['degree','closeness','betweenness','eigenvector','HITS','Katz','PageRank','load','communicability','current flow']
for cent in valus.values():
    cs={}
    for k,v in cent.items():
        if v not in cs:
            cs[v]=[k]
        else:
            cs[v].append(k)
    nodrank=[]
    uui=0
    for k in sorted(cs,reverse=True):
        for v in cs[k]:
            if uui<len(G_cl): #5:
                nodrank.append(v)
                uui+=1
    nodeclo=[]
    for k,v in cent.items():
        if k in  nodrank :
            nodeclo.append(v)
        else:
            nodeclo.append(0.)
plt.figure(figsize=(25,50))
for i,j in enumerate(cts): 
    plt.subplot(len(cts)/2,2,i+1).set_title(j)
    nx.draw_networkx_nodes(G_cl,pos=pos,nodelist=valus[j].keys(),
                               node_size=[500*x for x in valus[j].values()],
                               node_color=nodeclo,
                               cmap=plt.cm.cool,alpha=0.7) #Reds
    nx.draw_networkx_edges(G_cl,pos=pos,edge_color='b', alpha=0.5)
    y_off = 50 #0.05  # offset on the y axis
    pos_off = {key: (pos[key][0],pos[key][1] - y_off) for key in pos}
    nx.draw_networkx_labels(G_cl,pos=pos_off,font_weight='bold',font_size='10');
    nx.draw_networkx_edges(G_cl,pos=pos,edge_color='b', alpha=0.5)
    plt.title(j,fontsize=20)
    kk=plt.axis('off')
sst="%s Centrality Indices" %name
plt.suptitle(sst,fontsize=20);

In [0]:
comm_dic = nx.communicability(G_cl)
comm_dic2 = {}
for k,z in comm_dic.items():
  comm_dic2[k] = {}
  comm_dic2[k]['communicability'] = {}
  data_list = []
  for k1,z1 in z.items():
    data_list.append(z1)
  comm_dic2[k]['communicability'] = round(sum(data_list)/len(data_list),3)
  sent_avg = []
  widths2 = []
  for e in e_dic_cl.keys():
    if (k in e[0]) or (k in e[1]):
      sent_avg.append(e_dic_cl[e]['sentiment']*e_dic_cl[e]['width'])
      widths2.append(e_dic_cl[e]['width'])
  comm_dic2[k]['avg_sentiment'] = round(sum(sent_avg)/(sum(widths2)*len(sent_avg)),3)
comm_dic2

In [0]:
yd = [comm_dic2[k]['avg_sentiment'] for k,z in comm_dic2.items()]
xd = [comm_dic2[k]['communicability'] for k,z in comm_dic2.items()]

plt.figure(figsize=(10,10))

# make the scatter plot
plt.scatter(xd, yd, s=30, alpha=0.75, marker='o')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)
plt.text(.4*max(xd)+.1*min(xd),0.2*max(yd)+.1*min(yd),'$R^2 = %0.2f$'% Rsqr, fontsize=30)

plt.ylabel("Weighted Sentiment",size=20)
plt.xlabel("Communicability",size=20)
plt.title('Weighted Sentiment vs. Communicability',size=25)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

# yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
# yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

for k,z in comm_dic.items():
  y,x = comm_dic2[k]['avg_sentiment'],comm_dic2[k]['communicability']
  if (abs(y) > 0.08) or (k=='Dutch'):
    plt.text(x+.01,y+.02, k, fontsize=12,fontweight='bold')

plt.plot(xl, yl, '-r')
plt.axhline(y=0, color='k', linestyle='-')
# plt.plot(xd, yerrLower, '--r')
# plt.plot(xd, yerrUpper, '--r')
plt.show()

In [0]:
xd_list = [i/(len(G.nodes)*len(G.edges)) for i in xd]
xd_list = [(i-min(xd_list)) for i in xd_list]
#xd_list = xd
yd_list = yd
xd_list

In [0]:
import scipy
scipy.stats.pearsonr(xd, yd)

In [0]:
clos_dic = nx.closeness_centrality(G_cl)

In [0]:
yd = [comm_dic2[k]['avg_sentiment'] for k,z in clos_dic.items()]
xd = [clos_dic[k] for k,z in clos_dic.items()]

plt.figure(figsize=(10,10))

# make the scatter plot
plt.scatter(xd, yd, s=30, alpha=0.75, marker='o')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)
plt.text(.7*max(xd)+.1*min(xd),0.25*max(yd)+.1*min(yd),'$R^2 = %0.2f$'% Rsqr, fontsize=30)

plt.ylabel("Weighted Sentiment",size=20)
plt.xlabel("Closeness",size=20)
plt.title('Weighted Sentiment vs. Closeness',size=25)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

# yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
# yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

for k,z in clos_dic.items():
  y,x = comm_dic2[k]['avg_sentiment'],clos_dic[k]
  if (abs(y) > 0.08) or k=='Dutch':
    plt.text(x+.005,y+.025, k, fontsize=12,fontweight='bold')

plt.plot(xl, yl, '-r')
plt.axhline(y=0, color='k', linestyle='-')
# plt.plot(xd, yerrLower, '--r')
# plt.plot(xd, yerrUpper, '--r')
plt.show()

In [0]:
xd = [(i-min(xd)) for i in xd]
xd_list = xd_list + xd
yd_list = yd_list + yd

In [0]:
load_dic = nx.load_centrality(G_cl,normalized=True)

In [0]:
load_dic

In [0]:
yd = [comm_dic2[k]['avg_sentiment'] for k,z in load_dic.items()]
xd = [load_dic[k] for k,z in load_dic.items()]

plt.figure(figsize=(10,10))

# make the scatter plot
plt.scatter(xd, yd, s=30, alpha=0.75, marker='o')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)
plt.text(.5*max(xd)+.1*min(xd),0.25*max(yd)+.1*min(yd),'$R^2 = %0.2f$'% Rsqr, fontsize=30)

plt.ylabel("Weighted Sentiment",size=20)
plt.xlabel("Load",size=20)
plt.title('Weighted Sentiment vs. Load',size=25)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

# yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
# yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

for k,z in load_dic.items():
  y,x = comm_dic2[k]['avg_sentiment'],load_dic[k]
  if (abs(y) > 0.08) and k!='French' or k=='Dutch':
    plt.text(x+0.003,y+.0025, k, fontsize=10,fontweight='bold')
  if (abs(y) > 0.08) and k=='French':
    plt.text(x+0.003,y-.015, k, fontsize=10,fontweight='bold')

plt.plot(xl, yl, '-r')
plt.axhline(y=0, color='k', linestyle='-')
# plt.plot(xd, yerrLower, '--r')
# plt.plot(xd, yerrUpper, '--r')
plt.show()

In [0]:
xd_list = xd_list + xd
yd_list = yd_list + yd

In [0]:
from matplotlib.pyplot import cm
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

xlst=xd_list
ylst=yd_list
n=int(len(xd_list)/3)
xd_list1 = [xlst[i:i + n] for i in range(0, len(xlst), n)]
yd_list1 = [ylst[i:i + n] for i in range(0, len(ylst), n)]

max_val = max([max(i) for i in xd_list1])

i = 0
while i < len(xd_list1):
  this_max = max(xd_list1[i])
  if this_max != max_val:
    xd_list1[i] = [j*max_val/this_max for j in xd_list1[i]]
  i += 1

plt.figure(figsize=(10,10))

arr = []
colors = cm.viridis_r(np.linspace(0, 1, len(yd_list1)))

j_ind = list(load_dic.keys()).index('Jew')

i = 0
s_sizes = [400,200,75]
for x,y,c in zip(xd_list1,yd_list1, colors):
  arr.append(plt.scatter(x, y, color=c,marker='o',s=s_sizes[i]))
  i += 1

xdj_list1 = [el[j_ind] for el in xd_list1]
ydj_list1 = [el[j_ind] for el in yd_list1]

i = 0
for x,y,c in zip(xdj_list1,ydj_list1, colors):
  arr.append(plt.scatter(x, y, color=c,marker='X',s=s_sizes[i]*4.5))
  i += 1

par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

plt.ylabel("Weighted Sentiment",size=20)
plt.xlabel("Relative Centrality",size=20)
plt.title('Weighted Sentiment vs. Centrality',size=25)

yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

plt.legend(arr,('Communicability','Closeness','Load'),prop={'size': 15})

plt.axhline(y=0, color='k', linestyle='-')

plt.show()

In [0]:
yd = [comm_dic2[k]['avg_sentiment'] for k,z in load_dic.items()]
xd = [load_dic[k] for k,z in load_dic.items()]

plt.figure(figsize=(10,10))

# make the scatter plot
plt.scatter(xd, yd, s=30, alpha=0.75, marker='o')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)
plt.text(.5*max(xd)+.1*min(xd),0.25*max(yd)+.1*min(yd),'$R^2 = %0.2f$'% Rsqr, fontsize=30)

plt.ylabel("Weighted Sentiment",size=20)
plt.xlabel("Load",size=20)
plt.title('Weighted Sentiment vs. Load',size=25)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

# yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
# yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

for k,z in load_dic.items():
  y,x = comm_dic2[k]['avg_sentiment'],load_dic[k]
  if (abs(y) > 0.08) and k!='French' or k=='Dutch':
    plt.text(x+0.003,y+.0025, k, fontsize=10,fontweight='bold')
  if (abs(y) > 0.08) and k=='French':
    plt.text(x+0.003,y-.015, k, fontsize=10,fontweight='bold')

plt.plot(xl, yl, '-r')
plt.axhline(y=0, color='k', linestyle='-')
# plt.plot(xd, yerrLower, '--r')
# plt.plot(xd, yerrUpper, '--r')
plt.show()

In [0]:
yd = [comm_dic2[k]['avg_sentiment'] for k,z in load_dic.items()]
xd = [load_dic[k] for k,z in load_dic.items()]

plt.figure(figsize=(10,10))

# make the scatter plot
plt.scatter(xd, yd, s=30, alpha=0.75, marker='o')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)
plt.text(.5*max(xd)+.1*min(xd),0.25*max(yd)+.1*min(yd),'$R^2 = %0.2f$'% Rsqr, fontsize=30)

plt.ylabel("Weighted Sentiment",size=20)
plt.xlabel("Load",size=20)
plt.title('Weighted Sentiment vs. Load',size=25)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

# yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
# yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

for k,z in load_dic.items():
  y,x = comm_dic2[k]['avg_sentiment'],load_dic[k]
  if (abs(y) > 0.08) and k!='French' or k=='Dutch':
    plt.text(x+0.003,y+.0025, k, fontsize=10,fontweight='bold')
  if (abs(y) > 0.08) and k=='French':
    plt.text(x+0.003,y-.015, k, fontsize=10,fontweight='bold')

plt.plot(xl, yl, '-r')
plt.axhline(y=0, color='k', linestyle='-')
# plt.plot(xd, yerrLower, '--r')
# plt.plot(xd, yerrUpper, '--r')
plt.show()

###10b. Density

In [0]:
widths = [e_dic_cl[i]['width'] for i in G_cl.edges]
maxw = max(widths)
n = len(widths)
weighted_edge = round(sum(widths)/(n*(n-1)/2*maxw),5)
print('The Density of G is: {}'.format(round(nx.density(G_cl),3)))
print('The Weighted Density of G is: {}'.format(weighted_edge))

###10c. Degree Histogram

In [0]:
def weight_attr(G_cl,maxw):
    weight={}
    for edge in G_cl.edges():
        ed=edge[0]
        de=edge[1]
        G_cl[ed][de]['weight']=random.randrange(1,maxw)
        weight[(ed,de)]=G_cl[ed][de]['weight']
    return weight

In [0]:
from pylab import hist

ds=dict(G_cl.degree(G_cl.nodes())).values()
bins=7

plt.figure(figsize=(20,7))
plt.subplot(121)
plt.xlabel("Degree");
hist(ds,bins=bins,color='g'); 
# histtype='step',
plt.ylabel("Number of nodes");
ss='Degree Histogram of G' 
plt.title(ss,fontsize=20);
plt.subplot(122)
plt.xlabel("Degree");
hist(ds,bins=bins,color='g',log=True); 
# histtype='step',
plt.ylabel("Number of nodes");
sss='Log Degree Histogram of G' 
plt.title(sss,fontsize=20);

In [0]:
k = []
Pk = []

for node in list(G_cl.nodes()):
    degree = G_cl.degree(nbunch=node)
    try:
        post = k.index(degree)
    except ValueError as e:
        k.append(degree)
        Pk.append(1)
    else:
        Pk[post] += 1

logk=[]
logPk=[]
# get a double log representation
for i in range(len(k)):
    if k[i]>0:
        logk.append(math.log10(k[i]))
        logPk.append(math.log10(Pk[i]))

order = np.argsort(logk)
logk_array = np.array(logk)[order]
logPk_array = np.array(logPk)[order]
plt.plot(logk_array, logPk_array, ".")
m, c = np.polyfit(logk_array, logPk_array, 1)
print("m =",m, "c =",c)
plt.plot(logk_array, m*logk_array + c, "-");

if m>-3 and m<-2:
    print("The %s is scale-free" %name)
else:
    print("The %s is not scale-free" %name)

In [0]:
dsw=dict(G_cl.degree(G_cl.nodes()),weight='weight').values()
bins=7

plt.figure(figsize=(20,7))
plt.subplot(121)
plt.xlabel("Weighted Degree");
hist(dsw,bins=bins,color='g'); 
# histtype='step',
plt.ylabel("Number of nodes");
ss='Weighted Degree Histogram of G' 
plt.title(ss,fontsize=20);
plt.subplot(122)
plt.xlabel("Weighted Degree");
hist(dsw,bins=bins,color='g',log=True); 
# histtype='step',
plt.ylabel("Number of nodes");
sss='Log Weighted Degree Histogram of G' 
plt.title(sss,fontsize=20);

In [0]:
k = []
Pk = []

for node in list(G_cl.nodes()):
    degree = G_cl.degree(nbunch=node,weight='weight')
    try:
        post = k.index(degree)
    except ValueError as e:
        k.append(degree)
        Pk.append(1)
    else:
        Pk[post] += 1

logk=[]
logPk=[]
# get a double log representation
for i in range(len(k)):
    if k[i]>0:
        logk.append(math.log10(k[i]))
        logPk.append(math.log10(Pk[i]))

order = np.argsort(logk)
logk_array = np.array(logk)[order]
logPk_array = np.array(logPk)[order]
plt.plot(logk_array, logPk_array, ".")
m, c = np.polyfit(logk_array, logPk_array, 1)
print("m =",m, "c =",c)
plt.plot(logk_array, m*logk_array + c, "-");

if m>-3 and m<-2:
    print("The wighted %s is scale-free" %name)
else:
    print("The weighted %s is not scale-free" %name)

###10d. Connectivity

In [0]:
if nx.is_connected(G_cl)==True:
    print ("This graph is a connected graph")
else:
    print ("This graph is a disconnected graph and it has",nx.number_connected_components(sGw),"connected components" )   
    giant = max(nx.connected_component_subgraphs(G_cl), key=len)
    sGwlcc=sGw.subgraph(giant)
    print ("The largest connected component of this graph is a weighted graph with %i nodes and %i edges" %(len(sGwlcc.nodes()),len(sGwlcc.edges())))
    print ("The density of the largest connected component of this graph is %.3f" %nx.density(sGwlcc))


###10e. Transitivity

In [0]:
print('The transitivity of G is: {}'.format(round(nx.transitivity(G_cl),3)))
#Compute graph transitivity, the fraction of all possible triangles present in G.
#Possible triangles are identified by the number of “triads” (two edges with a shared vertex).

###10f. Clustering

In [0]:
Gt = nx.Graph()
wd = []
Gt.add_nodes_from(G_cl.nodes)
for edge in G_cl.edges:
  wd.append((edge[0],edge[1],e_dic_cl[edge]['width']))
Gt.add_weighted_edges_from(wd)
print('The weighted clustering coefficients are:')
clusdic = nx.clustering(Gt,weight='weight')
for z,v in clusdic.items():
  clusdic[z] = round(v,3)
clusdic

In [0]:
print('The average weighted clustering is: {}'.format(round(nx.average_clustering(Gt, nodes=None, weight='weight', count_zeros=True),3)))

###10g. Distances

In [0]:
print('The diamaeter of G is: {}'.format(nx.diameter(Gt)))

In [0]:
print('The average shortest path of G is: {}'.format(round(nx.average_shortest_path_length(Gt),2)))
print('The weighted average shortest path of G is: {}'.format(round(nx.average_shortest_path_length(Gt, weight='weight'),2)))

###10g. Assortativity

In [0]:
print('The weighted degree assortativity coefficient is: {}'.format(round(nx.degree_assortativity_coefficient(Gt, x='out', y='in', weight='weight', nodes=None),2)))

In [0]:
print('The attribute assortativity coefficient is: {}'.format(get_att_assort(Gt,n_dic_cl)))

In [0]:
def make_mapping(G,n_dic,att='tag'):
  mapping = {}
  i = 0
  for node in G.nodes:
    if n_dic[node][att] not in mapping.keys():
      mapping[n_dic[node][att]] = i
      i += 1
  return mapping
nx.set_node_attributes(Gt, {z:n_dic_cl[z]['tag'] for z,v in n_dic.items()}, 'tag')
mapping = make_mapping(Gt,n_dic_cl,att='tag')
mix_mat = nx.attribute_mixing_matrix(Gt, 'tag',mapping=mapping)

In [0]:
from matplotlib import colors
print('Mixing matrix of G by NLP tag:')
df_mix = pd.DataFrame(data=mix_mat,index=mapping.keys(),columns=mapping.keys())*len(Gt.edges)*2
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

df_mix.style.apply(background_gradient,
               cmap='cool',
               m=df_mix.min().min(),
               M=df_mix.max().max(),
               low=0,
               high=0.1)